In [ ]:
# ================================================================
# CELDA 0 — Setup reproducible (IDS2018, Colab-friendly)
#   • Monta Google Drive (si existe)
#   • Fija BASE y carpetas de trabajo
#   • Semillas y huella de versiones
#   • settings.json persistente (seed, label_col, key_strategy)
# ================================================================
import os, sys, json, random, platform, importlib
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd

def log(m): print(f"[Celda0] {datetime.now():%H:%M:%S} {m}", flush=True)

# --- Montar Drive (Colab) ---
try:
    from google.colab import drive  # type: ignore
    drive.mount("/content/drive", force_remount=True)
except Exception as e:
    log(f"Aviso al montar Drive (ok si no estás en Colab): {e}")

# -- MÍNIMO AÑADIDO: fijar IDS_BASE para que lo use la Celda 6/7
os.environ["IDS_BASE"] = "/content/drive/MyDrive/TFG/DATASETS/IDS2018"

# --- Rutas fijas (NO cambiar en celdas posteriores) ---
BASE = Path("/content/drive/MyDrive/TFG/DATASETS/IDS2018").resolve()
assert BASE.exists(), f"La ruta BASE no existe: {BASE}"

PREP         = BASE / "prepared"
PARQUET_DIR  = PREP / "parquet_parts"
MODELS_DIR   = BASE / "models_ids2018"
REPORTS_DIR  = BASE / "reports_ids2018"
EXPORT_DIR   = BASE / "export"

for d in (PREP, PARQUET_DIR, MODELS_DIR, REPORTS_DIR, EXPORT_DIR):
    d.mkdir(parents=True, exist_ok=True)

# --- settings.json persistente ---
CONFIG = PREP / "settings.json"
cfg = {"dataset":"CIC-IDS2018","seed":42,"label_col":"label","key_strategy":"auto"}
if CONFIG.exists():
    try:
        saved = json.loads(CONFIG.read_text())
        cfg.update(saved)  # permite cambiar seed o label_col y que persista
    except Exception as e:
        log(f"Aviso leyendo settings.json: {e}")
CONFIG.write_text(json.dumps(cfg, indent=2))

# --- Reproducibilidad (usa seed de settings.json) ---
SEED = int(cfg.get("seed", 42))
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

# --- Huella de versiones (para trazabilidad) ---
VERS = {
    "python": sys.version.split()[0],
    "platform": platform.platform(),
    "pandas": pd.__version__,
    "numpy": np.__version__,
}
for pkg in ["pyarrow","sklearn"]:
    try:
        VERS[pkg] = importlib.import_module(pkg).__version__
    except Exception:
        VERS[pkg] = "(missing)"
(PREP / "versions.json").write_text(json.dumps(VERS, indent=2))

# --- Metadatos mínimos (estado del entorno) ---
env_meta = {
    "dataset": cfg["dataset"],
    "seed": SEED,
    "python": VERS["python"],
    "pandas": VERS["pandas"],
    "numpy": VERS["numpy"],
    "pyarrow": VERS.get("pyarrow"),
    "sklearn": VERS.get("sklearn"),
    "platform": VERS["platform"],
    "base": str(BASE),
    "prepared": str(PREP),
    "parquet_parts": str(PARQUET_DIR),
    "models_dir": str(MODELS_DIR),
    "reports_dir": str(REPORTS_DIR),
    "export_dir": str(EXPORT_DIR),
    "ts": datetime.now().isoformat(timespec="seconds"),
}
(PREP / "_ENV_IDS2018.json").write_text(json.dumps(env_meta, indent=2))

# --- Estado rápido ---
log(f"BASE       = {BASE}")
log(f"PREP       = {PREP}")
log(f"PARTS      = {PARQUET_DIR}")
log(f"MODELS_DIR = {MODELS_DIR}")
log(f"REPORTS    = {REPORTS_DIR}")
log(f"EXPORT     = {EXPORT_DIR}")
log(f"Semilla    = {SEED}")
log(f"Entorno    = pandas {VERS['pandas']} | numpy {VERS['numpy']} | pyarrow {VERS.get('pyarrow')} | sklearn {VERS.get('sklearn')}")
log("✅ Celda 0 lista. Sigue con la Celda 1.")

Mounted at /content/drive
[Celda0] 21:38:34 BASE       = /content/drive/MyDrive/TFG/DATASETS/IDS2018
[Celda0] 21:38:34 PREP       = /content/drive/MyDrive/TFG/DATASETS/IDS2018/prepared
[Celda0] 21:38:34 PARTS      = /content/drive/MyDrive/TFG/DATASETS/IDS2018/prepared/parquet_parts
[Celda0] 21:38:34 MODELS_DIR = /content/drive/MyDrive/TFG/DATASETS/IDS2018/models_ids2018
[Celda0] 21:38:34 REPORTS    = /content/drive/MyDrive/TFG/DATASETS/IDS2018/reports_ids2018
[Celda0] 21:38:34 EXPORT     = /content/drive/MyDrive/TFG/DATASETS/IDS2018/export
[Celda0] 21:38:34 Semilla    = 42
[Celda0] 21:38:34 Entorno    = pandas 2.2.2 | numpy 2.0.2 | pyarrow 18.1.0 | sklearn 1.6.1
[Celda0] 21:38:34 ✅ Celda 0 lista. Sigue con la Celda 1.


In [ ]:
# ================================================================
# CELDA 1 — Setup y rutas + localización de CSVs → raw_manifest.json
#   · Define/crea PREP si no existe
#   · Busca CSVs (por defecto recursivo bajo BASE o cwd)
#   · Guarda PREP/raw_manifest.json para Celda 1.5
#   · Deja alias csv_files / CSV_FILES en memoria (comodidad)
# ================================================================
from pathlib import Path
import json
import os

def ts(): import time; return time.strftime("%H:%M:%S")
def log(m): print(f"[Celda1] {ts()} {m}", flush=True)

# --- PREP: carpeta de trabajo donde irán artefactos ---
if 'PREP' not in globals() or PREP is None:
    PREP = Path.cwd() / "prep_ids2018"
PREP = Path(PREP)
(PREP / "manifests").mkdir(parents=True, exist_ok=True)
log(f"PREP = {PREP.resolve()}")

# --- Raíces de búsqueda de CSVs ---
SEARCH_ROOTS = []
if 'BASE' in globals() and BASE:
    SEARCH_ROOTS.append(Path(BASE))
if 'DATA_DIR' in globals() and DATA_DIR:
    SEARCH_ROOTS.append(Path(DATA_DIR))
if not SEARCH_ROOTS:
    SEARCH_ROOTS = [Path.cwd()]  # por defecto, carpeta actual

# --- Patrones útiles para IDS2018 (puedes ampliar si quieres) ---
INCLUDE_PATTERNS = [
    "*TrafficForML*CICFlowMeter*.csv",  # clásico de IDS2018
    "*CICFlowMeter*.csv",
    "*.csv"
]
EXCLUDE_SNIPPETS = [
    "parquet_parts", "features", "splits", "reports", "models",
    "keys_split_map", "salt_pick", "split_meta"
]

# --- Buscar CSVs (recursivo) ---
def find_csvs(roots):
    seen = set()
    results = []
    for root in roots:
        root = Path(root)
        if not root.exists():
            continue
        for pat in INCLUDE_PATTERNS:
            for p in root.rglob(pat):
                if not p.is_file() or p.suffix.lower() != ".csv":
                    continue
                sp = str(p).lower()
                if any(bad in sp for bad in EXCLUDE_SNIPPETS):
                    continue
                if p.resolve() in seen:
                    continue
                seen.add(p.resolve())
                results.append(p.resolve())
    return sorted(results)

csv_paths = find_csvs(SEARCH_ROOTS)
assert csv_paths, f"No se encontraron CSVs bajo {', '.join(str(r) for r in SEARCH_ROOTS)}"

# --- Guardar manifest bruto ---
raw_manifest = [{"path": str(p), "name": p.name, "size_bytes": p.stat().st_size} for p in csv_paths]
(PREP / "raw_manifest.json").write_text(json.dumps(raw_manifest, indent=2))

# --- Alias en memoria (comodidad; 1.5 leerá del manifest igualmente) ---
csv_files = [str(p) for p in csv_paths]
CSV_FILES = [Path(p) for p in csv_paths]

log(f"Localizados {len(csv_paths)} CSV(s)")
for p in csv_paths:
    log(f"  · {p}")
log("✔️ Escrito PREP/raw_manifest.json")

[Celda1] 21:38:36 PREP = /content/drive/MyDrive/TFG/DATASETS/IDS2018/prepared
[Celda1] 21:38:39 Localizados 5 CSV(s)
[Celda1] 21:38:39   · /content/drive/MyDrive/TFG/DATASETS/IDS2018/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
[Celda1] 21:38:39   · /content/drive/MyDrive/TFG/DATASETS/IDS2018/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
[Celda1] 21:38:39   · /content/drive/MyDrive/TFG/DATASETS/IDS2018/Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
[Celda1] 21:38:39   · /content/drive/MyDrive/TFG/DATASETS/IDS2018/prepared/train_full.csv
[Celda1] 21:38:39   · /content/drive/MyDrive/TFG/DATASETS/IDS2018/prepared/val_full.csv
[Celda1] 21:38:39 ✔️ Escrito PREP/raw_manifest.json


In [ ]:
# ================================================================
# CELDA 1.5 — Lectura/Limpieza + clave → parquet + manifest (fast-path)
# ================================================================
import json, gc
from pathlib import Path
import pandas as pd
import numpy as np
import time
import os

def ts(): return time.strftime("%H:%M:%S")
def log(m): print(f"[Celda1.5] {ts()} {m}", flush=True)

assert 'PREP' in globals(), "Ejecuta antes la CELDA 0 (define PREP)."
PREP = Path(PREP)
PARQUET_DIR = PREP / "parquet_parts"; PARQUET_DIR.mkdir(parents=True, exist_ok=True)
MANIFESTS   = PREP / "manifests";     MANIFESTS.mkdir(parents=True, exist_ok=True)

prepared_manifest_path = MANIFESTS / "prepared_manifest.json"
diag_path = PREP / "_DIAG_DATA_COLS.json"
RAW_MANIFEST = PREP / "raw_manifest.json"

# ---------- FAST-PATH ----------
FORCE_REBUILD = False  # Pon True solo si se quiere volver a procesar CSVs
if prepared_manifest_path.exists() and diag_path.exists() and not FORCE_REBUILD:
    man = json.loads(prepared_manifest_path.read_text())
    key_cols = json.loads(diag_path.read_text()).get("KEY_COLS", ["srcip","sport","dstip","dsport","proto"])
    PARQUET_DIR = Path(man.get("parquet_dir", PARQUET_DIR))
    # Reconstruye CSV_FILES por compatibilidad (no imprescindible)
    if RAW_MANIFEST.exists():
        _raw = json.loads(RAW_MANIFEST.read_text())
        CSV_FILES = [Path(it["path"]) for it in _raw]
    log(f"Reutilizando artefactos existentes → partes={man['n_parts']} filas={man['n_rows']:,} | KEY_COLS={key_cols}")
else:
    # ---------- PROCESADO COMPLETO ----------
    assert RAW_MANIFEST.exists(), "Falta PREP/raw_manifest.json. Ejecuta la CELDA 1."
    _manifest = json.loads(RAW_MANIFEST.read_text())
    CSV_FILES = [Path(it["path"]) for it in _manifest]
    log(f"Cargados {len(CSV_FILES)} CSV(s) desde raw_manifest.json")

    # Normalización de nombres
    def norm_colname(c: str) -> str:
        c = str(c).strip().lower()
        for ch in [' ', '\t', '\n', '\r', '-', '.', '/', '\\']:
            c = c.replace(ch, '_')
        while '__' in c: c = c.replace('__','_')
        return c.strip('_')
    def normalize_df_cols(df: pd.DataFrame) -> pd.DataFrame:
        df.columns = [norm_colname(c) for c in df.columns]; return df

    # Sinónimos y helpers
    CAND = {
        "srcip":  ["srcip","source_ip","src_ip","sourceip","src_address","srcaddr","ip_src","id_orig_h","sourceaddress"],
        "dstip":  ["dstip","destination_ip","dst_ip","destinationip","dst_address","dstaddr","ip_dst","id_resp_h","destinationaddress"],
        "sport":  ["sport","source_port","src_port","srcport","tcp_srcport","udp_srcport","id_orig_p","src_prt"],
        "dsport": ["dsport","destination_port","dst_port","dstport","tcp_dstport","udp_dstport","id_resp_p","dst_prt"],
        "proto":  ["proto","protocol","protocol_name","protocoltype","ip_proto","transport","proto_num"],
        "label":  ["label","class","attack_cat","attack","category","target","malicious"],
        "flow_id":["flow_id","flowid","fid","flow___id","flow__id","flow_id_"],
        "timestamp":    ["timestamp","flow_start","start_time","starttimestamp","time_start"],
        "flow_duration":["flow_duration","flowduration","flow_dur","duration"],
        "tot_fwd_pkts": ["tot_fwd_pkts","totalfwdpackets","fwd_pkts_tot","fwd_iat_tot"],
        "tot_bwd_pkts": ["tot_bwd_pkts","totalbwdpackets","bwd_pkts_tot","bwd_iat_tot"],
    }
    def pick_first(df, opts):
        for o in opts:
            if o in df.columns: return o
        return None
    def binarize_label(s: pd.Series) -> pd.Series:
        if pd.api.types.is_numeric_dtype(s):
            return (pd.to_numeric(s, errors='coerce').fillna(0) != 0).astype(np.int8)
        st = s.astype(str).str.strip().str.lower()
        y = ~st.isin(("benign","normal","background"))
        y = y & ~(st.isin(("0","false","none","nan","")))
        return y.astype(np.int8)
    def proto_to_iana(s: pd.Series) -> pd.Series:
        if pd.api.types.is_numeric_dtype(s):
            return pd.to_numeric(s, errors='coerce').astype('Int64')
        st = s.astype(str).str.strip().str.lower()
        return pd.Series(np.select([
            st.str.contains("tcp",  na=False),
            st.str.contains("udp",  na=False),
            st.str.contains("icmp", na=False),
        ], [6,17,1], default=np.nan), index=s.index).astype('Int64')
    def proto_name_or_num_to_iana(x):
        try: return int(x)
        except Exception:
            s = str(x).strip().lower()
            if "tcp" in s:  return 6
            if "udp" in s:  return 17
            if "icmp" in s: return 1
            return np.nan
    def try_parse_flow_id(df, flow_col: str):
        flow = df[flow_col].astype(str).str.strip()
        parts = flow.str.split('-', n=4, expand=True)  # 5 campos
        if parts.shape[1] != 5: return None
        srcip  = parts[0].astype(str)
        dstip  = parts[1].astype(str)
        sport  = pd.to_numeric(parts[2], errors='coerce')
        dsport = pd.to_numeric(parts[3], errors='coerce')
        proto  = parts[4].map(proto_name_or_num_to_iana).astype('Int64')
        return {"srcip": srcip, "dstip": dstip, "sport": sport, "dsport": dsport, "proto": proto}
    def make_flow_key(df, c_time, c_dport, c_proto, c_dur, c_fwd, c_bwd) -> pd.Series:
        ts   = df[c_time].astype(str).fillna("")
        dpt  = pd.to_numeric(df[c_dport], errors='coerce').fillna(-1).astype(np.int64)
        pr   = proto_to_iana(df[c_proto]).fillna(-1).astype(np.int64)
        dur  = pd.to_numeric(df[c_dur], errors='coerce').fillna(-1).astype(np.int64)
        fwd  = pd.to_numeric(df[c_fwd], errors='coerce').fillna(-1).astype(np.int64)
        bwd  = pd.to_numeric(df[c_bwd], errors='coerce').fillna(-1).astype(np.int64)
        tmp = pd.DataFrame({"ts":ts, "dpt":dpt, "pr":pr, "dur":dur, "fwd":fwd, "bwd":bwd})
        return tmp.astype(str).agg('|'.join, axis=1)

    CHUNKSIZE = 250_000
    prepared_parts, FALLBACK_USED = [], False

    for csv in CSV_FILES:
        assert csv.exists(), f"No existe: {csv}"
        enc = "utf-8"
        try:
            _ = pd.read_csv(csv, nrows=5, encoding=enc, on_bad_lines='skip', low_memory=False)
        except Exception:
            enc = "latin-1"
        log(f"Leyendo {csv.name} (encoding={enc})")
        chunk_i = 0
        for df in pd.read_csv(csv, chunksize=CHUNKSIZE, encoding=enc,
                              on_bad_lines='skip', low_memory=False):
            chunk_i += 1; df = normalize_df_cols(df)

            # Intento 5-tupla
            c_srcip  = pick_first(df, CAND["srcip"])
            c_sport  = pick_first(df, CAND["sport"])
            c_dstip  = pick_first(df, CAND["dstip"])
            c_dsport = pick_first(df, CAND["dsport"])
            c_proto  = pick_first(df, CAND["proto"])
            c_label  = pick_first(df, CAND["label"])
            c_flowid = pick_first(df, CAND["flow_id"])

            used_flow_key = False

            # Fallback 1: flow_id
            if any(c is None for c in [c_srcip,c_sport,c_dstip,c_dsport,c_proto]) and c_flowid:
                recon = try_parse_flow_id(df, c_flowid)
                if recon is not None:
                    if c_srcip  is None: df["__recon_srcip__"]  = recon["srcip"];  c_srcip  = "__recon_srcip__"
                    if c_dstip  is None: df["__recon_dstip__"]  = recon["dstip"];  c_dstip  = "__recon_dstip__"
                    if c_sport  is None: df["__recon_sport__"]  = recon["sport"];  c_sport  = "__recon_sport__"
                    if c_dsport is None: df["__recon_dsport__"] = recon["dsport"]; c_dsport = "__recon_dsport__"
                    if c_proto  is None: df["__recon_proto__"]  = recon["proto"];  c_proto  = "__recon_proto__"
                    log(f"  ↪︎ reconstruidas columnas desde {c_flowid} en chunk {chunk_i}")

            # Fallback 2: flow_key
            if any(c is None for c in [c_srcip, c_sport, c_dstip]) or c_proto is None or c_dsport is None:
                c_time = pick_first(df, CAND["timestamp"])
                c_dur  = pick_first(df, CAND["flow_duration"])
                c_fwd  = pick_first(df, CAND["tot_fwd_pkts"])
                c_bwd  = pick_first(df, CAND["tot_bwd_pkts"])
                if all(x is not None for x in [c_time, c_dsport, c_proto, c_dur, c_fwd, c_bwd]):
                    df["flow_key"] = make_flow_key(df, c_time, c_dsport, c_proto, c_dur, c_fwd, c_bwd)
                    used_flow_key = True; FALLBACK_USED = True
                    log(f"  ↪︎ usando flow_key en chunk {chunk_i} (sin 5-tupla/flow_id)")
                else:
                    sample_cols = list(df.columns[:30])
                    raise AssertionError(
                        f"No hay columnas suficientes para construir clave (chunk {chunk_i}). "
                        "Necesarias: timestamp, dst_port, protocol, flow_duration, tot_fwd_pkts, tot_bwd_pkts\n"
                        f"Primeras columnas: {sample_cols}"
                    )

            # Salida
            out = df.copy()
            if not used_flow_key:
                out["srcip"]  = df[c_srcip].astype(str)
                out["dstip"]  = df[c_dstip].astype(str)
                out["sport"]  = pd.to_numeric(df[c_sport],  errors='coerce').fillna(-1).astype(np.int64)
                out["dsport"] = pd.to_numeric(df[c_dsport], errors='coerce').fillna(-1).astype(np.int64)
                out["proto"]  = proto_to_iana(df[c_proto]).fillna(-1).astype(np.int64)

            out["label"] = 0 if c_label is None else binarize_label(df[c_label]).astype(np.int8)

            out_name = f"{csv.stem}.part{chunk_i:03d}.parquet"
            out_path = PARQUET_DIR / out_name
            out.to_parquet(out_path, index=False, engine="pyarrow")
            prepared_parts.append({"part": out_name, "rows": int(len(out)), "src_csv": str(csv), "chunk_idx": int(chunk_i), "used_flow_key": bool(used_flow_key)})
            log(f"  → {out_name} | filas={len(out):,} | clave={'flow_key' if used_flow_key else '5tuple'}")

            del df, out; gc.collect()

    # Manifest + DIAG
    prepared_manifest = {
        "parquet_dir": str(PARQUET_DIR),
        "n_parts": len(prepared_parts),
        "n_rows": int(sum(p["rows"] for p in prepared_parts)),
        "parts": prepared_parts,
    }
    prepared_manifest_path.write_text(json.dumps(prepared_manifest, indent=2))
    key_cols = ["flow_key"] if any(p["used_flow_key"] for p in prepared_parts) else ["srcip","sport","dstip","dsport","proto"]
    diag = {"KEY_COLS": key_cols, "LABEL_COL": "label", "df_keys_cols": key_cols}
    diag_path.write_text(json.dumps(diag, indent=2))
    log(f"✔️ Guardado manifest y DIAG | KEY_COLS={key_cols} | filas={prepared_manifest['n_rows']:,} | partes={prepared_manifest['n_parts']}")

[Celda1.5] 21:39:06 Reutilizando artefactos existentes → partes=15 filas=3,145,725 | KEY_COLS=['flow_key']


In [ ]:
# ================================================
# CELDA 2 — Split por claves con salting y tiers (DEF)
#   · 70/15/15 sin fuga (hash por clave)
#   · Búsqueda vectorizada (NumPy) con progreso y presupuesto por tier
#   · Por defecto: reproducible con SALT de Tier 3 que ya te funcionó
#   · Artefactos: keys_split_map.parquet, split_meta.json, salt_pick.json
# ================================================
import os, json, math, time, hashlib, gc
from pathlib import Path
from typing import List
import numpy as np
import pandas as pd

# ---------- Log ----------
def ts(): return time.strftime("%H:%M:%S")
def log(msg): print(f"[Celda2] {ts()} {msg}", flush=True)

# ---------- Precondiciones ----------
assert 'BASE' in globals() and 'PREP' in globals(), "Ejecuta antes Celda 0/1."
BASE = Path(BASE); PREP = Path(PREP)
if 'PARQUET_DIR' not in globals(): PARQUET_DIR = PREP / "parquet_parts"
if 'MANIFESTS'   not in globals(): MANIFESTS   = PREP / "manifests"
PARQUET_DIR.mkdir(parents=True, exist_ok=True)

# ---------- Ratios ----------
RATIOS = {"train": 0.70, "val": 0.15, "test": 0.15}
NAME2CODE = {"train": 0, "val": 1, "test": 2}
CODE2NAME = {v: k for k, v in NAME2CODE.items()}

# ---------- Tiers (de más estricto a más laxo) ----------
TIERS = [
    {"name":"Tier 1","bins":240_000,"size_pp":5.0, "gap_pp":10.0,"range_pp":12.0,"salts":20_000},
    {"name":"Tier 2","bins":200_000,"size_pp":7.5, "gap_pp":12.0,"range_pp":15.0,"salts":20_000},
    {"name":"Tier 3","bins":160_000,"size_pp":10.0,"gap_pp":16.0,"range_pp":20.0,"salts":16_000},
    {"name":"Tier 4","bins":120_000,"size_pp":12.0,"gap_pp":20.0,"range_pp":25.0,"salts":18_000},
]

# ---------- Ajustes de búsqueda ----------
PROGRESS_EVERY   = 200          # cada cuántas sales imprimir progreso
TIME_BUDGET_SEC  = 60           # presupuesto por tier (sobrado si usarás OVERRIDE_SALT)
OVERRIDE_SALT    = 347222276    # ← congela nuestro SALT de Tier 3; poner None para buscar de nuevo

# ---------- Asignación ----------
# Usamos por defecto rangos contiguos (como en tu Tier 3 previo).
# Si quisieramos reintentar para bajar rango de prevalencias, poner STRIPED_ASSIGN=True y ajusta STRIPE.
STRIPED_ASSIGN       = False    # False = rangos contiguos (reproducir Tier 3)
INTERLEAVED_STRIPE   = 997      # solo si se activa STRIPED_ASSIGN
p_train = int(round(RATIOS["train"] * INTERLEAVED_STRIPE))
p_val   = int(round(RATIOS["val"]   * INTERLEAVED_STRIPE))

# ---------- Columnas clave ----------
key_cols_guess = ['srcip','sport','dstip','dsport','proto']
diag_path = PREP / "_DIAG_DATA_COLS.json"
if diag_path.exists():
    try:
        diag = json.loads(diag_path.read_text())
        if isinstance(diag.get('KEY_COLS'), list) and diag['KEY_COLS']:
            KEY_COLS = [c.lower() for c in diag['KEY_COLS']]
        elif isinstance(diag.get('df_keys_cols'), list) and diag['df_keys_cols']:
            KEY_COLS = [c.lower() for c in diag['df_keys_cols']]
        else:
            KEY_COLS = key_cols_guess
    except Exception:
        KEY_COLS = key_cols_guess
else:
    KEY_COLS = key_cols_guess
LABEL_COL = 'label'
log(f"Columnas de clave: {KEY_COLS}")

# ---------- Descubrimiento de partes (manifest + filtro de esquema) ----------
try:
    import pyarrow.parquet as pq
except Exception:
    pq = None

def _has_required_columns(p: Path, required: List[str]) -> bool:
    if pq is None:
        try:
            pd.read_parquet(p, columns=required, engine="pyarrow")
            return True
        except Exception:
            return False
    try:
        schema = pq.read_schema(str(p))
        cols = {f.name.lower() for f in schema}
        return set(map(str.lower, required)).issubset(cols)
    except Exception:
        return False

def discover_parts():
    required = [*KEY_COLS, LABEL_COL]
    parts = []
    man = MANIFESTS / "prepared_manifest.json"
    if man.exists():
        try:
            j = json.loads(man.read_text())
            for it in j.get("parts", []):
                p = (Path(j.get("parquet_dir", str(PARQUET_DIR))) / it["part"]).resolve()
                if p.is_file() and p.suffix == ".parquet" and _has_required_columns(p, required):
                    parts.append(p)
        except Exception:
            pass
    if not parts and PARQUET_DIR.exists():
        for p in sorted(PARQUET_DIR.glob("*.parquet")):
            name_low = p.name.lower()
            if any(k in name_low for k in ("keys_split_map","split_meta","salt_pick","features","scaler","idx_seq")):
                continue
            if _has_required_columns(p, required):
                parts.append(p)
    if not parts:
        for p in sorted(PREP.glob("*.parquet")):
            name_low = p.name.lower()
            if any(k in name_low for k in ("keys_split_map","split_meta","salt_pick","features","scaler","idx_seq")):
                continue
            if _has_required_columns(p, required):
                parts.append(p)
    return parts

# ---------- Resumen por clave (o cache) ----------
KEYS_SUMMARY = PREP / "_KEYS_COUNTS.parquet"

def normalize_cols(df):
    df.columns = df.columns.str.lower(); return df

def collect_key_counts_from_df(df, key_cols=KEY_COLS, label_col=LABEL_COL):
    df = normalize_cols(df)
    miss = [c for c in key_cols+[label_col] if c not in df.columns]
    if miss: raise AssertionError(f"Faltan columnas: {miss}")
    key_str = df[key_cols].astype(str).agg('|'.join, axis=1)
    y = pd.to_numeric(df[label_col], errors='coerce').fillna(0).astype(int)
    g = pd.DataFrame({"key": key_str, "y": y})
    pos = g.groupby("key")["y"].sum()
    cnt = g.groupby("key")["y"].count()
    res = pd.DataFrame({"key": pos.index, "pos": pos.values.astype(np.int64), "cnt": cnt.values.astype(np.int64)})
    res["neg"] = (res["cnt"] - res["pos"]).astype(np.int64)
    return res

parts = None
if KEYS_SUMMARY.exists():
    log("Usando resumen de claves precomputado (_KEYS_COUNTS.parquet)")
    keys_df = pd.read_parquet(KEYS_SUMMARY)
    keys_df["pos"] = keys_df["pos"].astype(np.int64)
    keys_df["neg"] = keys_df["neg"].astype(np.int64)
    keys_df["cnt"] = (keys_df["pos"] + keys_df["neg"]).astype(np.int64)
else:
    parts = discover_parts()
    assert parts, "No hay partes válidas con columnas requeridas. Ejecuta la Celda 1.5."
    log(f"Evaluando {len(parts)} partes…")
    acc = []
    for i, p in enumerate(parts, 1):
        if i == 1 or i % 6 == 0 or i == len(parts):
            log(f"  pre-pass {i}/{len(parts)}")
        dfp = pd.read_parquet(p, columns=[*KEY_COLS, LABEL_COL], engine="pyarrow")
        acc.append(collect_key_counts_from_df(dfp))
        del dfp; gc.collect()
    keys_df = pd.concat(acc, ignore_index=True)
    keys_df = keys_df.groupby("key", as_index=False).agg({"pos":"sum","cnt":"sum"})
    keys_df["neg"] = (keys_df["cnt"] - keys_df["pos"]).astype(np.int64)
    keys_df.to_parquet(KEYS_SUMMARY, index=False)

assert not keys_df.empty, "No hay claves tras el pre-pass."
N_KEYS   = len(keys_df)
TOT_ALL  = int(keys_df["cnt"].sum())
POS_ALL  = int(keys_df["pos"].sum())
PREV_GLOBAL = POS_ALL / TOT_ALL if TOT_ALL else 0.0
log(f"Resumen keys: N_KEYS={N_KEYS:,} | filas={TOT_ALL:,} | pos={POS_ALL:,} | prev_global={PREV_GLOBAL:.6f}")

# ---------- Helpers ----------
def size_deviation_pp(n_by_code, tot, ratios):
    # ratios por nombre; n_by_code por código (0,1,2)
    return {c: 100.0 * abs(((n_by_code.get(c,0))/float(tot)) - float(ratios[CODE2NAME[c]]))
            for c in (0,1,2)}

def prevalence_stats_from_masks(m0, m1, m2, pos_vec, neg_vec):
    pos0, neg0 = int(pos_vec[m0].sum()), int(neg_vec[m0].sum())
    pos1, neg1 = int(pos_vec[m1].sum()), int(neg_vec[m1].sum())
    pos2, neg2 = int(pos_vec[m2].sum()), int(neg_vec[m2].sum())
    agg = {0:{"pos":pos0,"neg":neg0}, 1:{"pos":pos1,"neg":neg1}, 2:{"pos":pos2,"neg":neg2}}
    prev = {c: (agg[c]["pos"]/max(1,(agg[c]["pos"]+agg[c]["neg"]))) for c in (0,1,2)}
    vals = list(prev.values())
    range_pp = (max(vals)-min(vals))*100.0
    gap_val_train_pp = abs(prev[1]-prev[0])*100.0
    n_by_code = {c: agg[c]["pos"]+agg[c]["neg"] for c in (0,1,2)}
    return agg, prev, n_by_code, range_pp, gap_val_train_pp

# ---------- Pre-cálculo vectorizado ----------
keys_arr = keys_df["key"].astype(str).values
pos_vec  = keys_df["pos"].to_numpy(dtype=np.int64)
cnt_vec  = keys_df["cnt"].to_numpy(dtype=np.int64)
neg_vec  = (cnt_vec - pos_vec).astype(np.int64)

# Hash base por clave (1 sola vez, 64 bits)
base_hash = np.array([
    int.from_bytes(hashlib.blake2b(k.encode('utf-8'), digest_size=8).digest(), 'little')
    for k in keys_arr
], dtype=np.uint64)
MIX_CONST = np.uint64(11400714819323198485)     # Knuth
MASK64    = np.uint64(0xFFFFFFFFFFFFFFFF)

MIN_POS_PER_SPLIT = 1

# ---------- Búsqueda de SALT (vectorizada) ----------
salt_pick = None
keys_to_split = None

for tier in TIERS:
    bins, size_hard, gap_hard, range_hard, max_salts = (
        tier["bins"], tier["size_pp"], tier["gap_pp"], tier["range_pp"], tier["salts"]
    )
    log(f"[{tier['name']}] bins={bins:,} | size±{size_hard:.1f}pp | gap≤{gap_hard:.1f}pp | range≤{range_hard:.1f}pp | salts={max_salts:,}")

    found = False
    rng = np.random.default_rng(42)
    if OVERRIDE_SALT is not None:
        salts_seq = [int(OVERRIDE_SALT)]
    else:
        structured = list(range(0, min(2000, max_salts)))
        randoms = rng.integers(1_000_000_000, size=max(0, max_salts-len(structured)))
        salts_seq = list(structured) + list(randoms)

    deadline = time.time() + TIME_BUDGET_SEC
    best = {"worst_dev_pp": float("inf"), "range_pp": float("inf"), "gap_vt": float("inf"), "salt": None}

    for trial, salt in enumerate(salts_seq, 1):
        if trial == 1 or (OVERRIDE_SALT is None and trial % PROGRESS_EVERY == 0):
            log(f"  probando salt {trial}/{max_salts}… (mejor: worst_dev={best['worst_dev_pp']:.3f}pp, range={best['range_pp']:.3f}pp, gapVT={best['gap_vt']:.3f}pp)")

        salt_u = np.uint64(int(salt))
        mix = (base_hash ^ salt_u) * MIX_CONST & MASK64
        bins_idx = (mix % np.uint64(bins)).astype(np.uint32)

        if STRIPED_ASSIGN:
            r = (bins_idx % np.uint32(INTERLEAVED_STRIPE)).astype(np.uint32)
            m0 = r < p_train
            m1 = (r >= p_train) & (r < p_train + p_val)
        else:
            t_bins = int(round(RATIOS["train"] * bins))
            v_bins = int(round((RATIOS["train"] + RATIOS["val"]) * bins))
            m0 = bins_idx < t_bins
            m1 = (bins_idx >= t_bins) & (bins_idx < v_bins)
        m2 = ~ (m0 | m1)

        agg, prev, n_by_code, range_pp, gap_val_train_pp = prevalence_stats_from_masks(m0, m1, m2, pos_vec, neg_vec)
        dev_size = size_deviation_pp(n_by_code, TOT_ALL, RATIOS)
        worst_dev_pp = max(dev_size.values()) if dev_size else 0.0

        # guarda mejor intento (diagnóstico)
        if (worst_dev_pp < best["worst_dev_pp"]) or (
            math.isclose(worst_dev_pp, best["worst_dev_pp"]) and (range_pp < best["range_pp"])
        ):
            best = {"worst_dev_pp": float(worst_dev_pp), "range_pp": float(range_pp),
                    "gap_vt": float(gap_val_train_pp), "salt": int(salt)}

        # checks del tier
        if any(pp > size_hard for pp in dev_size.values()):
            if time.time() > deadline: break
            continue
        if gap_val_train_pp > gap_hard or range_pp > range_hard:
            if time.time() > deadline: break
            continue
        if any(agg[c]["pos"] < MIN_POS_PER_SPLIT for c in (0,1,2)):
            if time.time() > deadline: break
            continue

        #válido!
        split_codes = np.full(N_KEYS, 2, dtype=np.int8)
        split_codes[m0] = 0
        split_codes[m1] = 1

        salt_pick = {
            "tier": tier,
            "salt": int(salt),
            "n_by_code": n_by_code,
            "prev_by_code": {CODE2NAME[c]: prev[c] for c in (0,1,2)},
            "metrics_extra": {"range_pp": float(range_pp), "gap_val_train_pp": float(gap_val_train_pp)},
            "dev_size_pp": {CODE2NAME[c]: float(dev_size[c]) for c in (0,1,2)},
            "worst_dev_pp": float(worst_dev_pp),
        }
        keys_to_split = split_codes
        found = True
        break

    if not found:
        log(f"  (sin solución en {tier['name']} dentro del presupuesto; mejor: "
            f"worst_dev={best['worst_dev_pp']:.3f}pp, range={best['range_pp']:.3f}pp, gapVT={best['gap_vt']:.3f}pp con SALT={best['salt']})")
    if found:
        break

assert salt_pick is not None, "No se ha encontrado un SALT que cumpla los tiers. Revisa límites o desactiva OVERRIDE_SALT para buscar."

# ---------- Guardado ----------
kmap = pd.DataFrame({"key": keys_df["key"].values, "split_code": keys_to_split.astype(np.int8)})
kmap["split"] = kmap["split_code"].map(lambda c: CODE2NAME[int(c)])
kmap.to_parquet(PREP / "keys_split_map.parquet", index=False)

split_meta = {
    "ratios_target": RATIOS,
    "prev_global": float(PREV_GLOBAL),
    "prev_global_pct": float(PREV_GLOBAL*100.0),
    "n_keys": int(N_KEYS), "n_rows": int(TOT_ALL),
    "key_cols": KEY_COLS, "label_col": LABEL_COL,
    "tier_used": salt_pick["tier"]["name"], "bins": int(salt_pick["tier"]["bins"]),
}
(PREP / "split_meta.json").write_text(json.dumps(split_meta, indent=2))

salt_dump = dict(salt_pick); salt_dump["tier"] = salt_pick["tier"]["name"]
(PREP / "salt_pick.json").write_text(json.dumps(salt_dump, indent=2))

# ---------- Resumen ----------
log("✔️ Artefactos creados: keys_split_map.parquet, split_meta.json, salt_pick.json")
log(f"Elegido: {salt_pick['tier']['name']} | SALT={salt_pick['salt']} | bins={split_meta['bins']:,}")
log(f"Tamaños (desvío pp): {salt_pick['dev_size_pp']}")
log(f"Prev por split: " + ", ".join(f"{name}={salt_pick['prev_by_code'][name]*100:.3f}%"
                                    for name in ("train","val","test")))
log(f"Rango prev (pp): {salt_pick['metrics_extra']['range_pp']:.3f} | Gap VAL-TRAIN (pp): {salt_pick['metrics_extra']['gap_val_train_pp']:.3f}")

[Celda2] 21:39:10 Columnas de clave: ['flow_key']
[Celda2] 21:39:10 Usando resumen de claves precomputado (_KEYS_COUNTS.parquet)
[Celda2] 21:39:11 Resumen keys: N_KEYS=34,956 | filas=3,145,725 | pos=1,035,250 | prev_global=0.329097
[Celda2] 21:39:11 [Tier 1] bins=240,000 | size±5.0pp | gap≤10.0pp | range≤12.0pp | salts=20,000
[Celda2] 21:39:11   probando salt 1/20000… (mejor: worst_dev=infpp, range=infpp, gapVT=infpp)
[Celda2] 21:39:11   (sin solución en Tier 1 dentro del presupuesto; mejor: worst_dev=18.248pp, range=34.570pp, gapVT=34.570pp con SALT=347222276)
[Celda2] 21:39:11 [Tier 2] bins=200,000 | size±7.5pp | gap≤12.0pp | range≤15.0pp | salts=20,000
[Celda2] 21:39:11   probando salt 1/20000… (mejor: worst_dev=infpp, range=infpp, gapVT=infpp)
[Celda2] 21:39:11   (sin solución en Tier 2 dentro del presupuesto; mejor: worst_dev=19.051pp, range=36.956pp, gapVT=36.956pp con SALT=347222276)
[Celda2] 21:39:11 [Tier 3] bins=160,000 | size±10.0pp | gap≤16.0pp | range≤20.0pp | salts=16,000

In [ ]:
# CELDA AUXILIAR === Validación del split (QA) ===
from pathlib import Path
from IPython.display import display, Markdown
import json, pandas as pd

assert 'PREP' in globals(), "Define PREP antes (mismo valor que en las celdas 1 y 2)."
PREP = Path(PREP)

# Carga de artefactos
kmap = pd.read_parquet(PREP / "keys_split_map.parquet")        # claves -> split
meta = json.loads((PREP / "split_meta.json").read_text())      # ratios, bins, etc.
salt = json.loads((PREP / "salt_pick.json").read_text())       # métricas del split

tier_name = salt.get("tier", meta.get("tier_used", "Tier 3"))
limits_by_tier = {
    "Tier 1": {"size_pp": 5.0,  "gap_pp": 10.0, "range_pp": 12.0},
    "Tier 2": {"size_pp": 7.5,  "gap_pp": 12.0, "range_pp": 15.0},
    "Tier 3": {"size_pp": 10.0, "gap_pp": 16.0, "range_pp": 20.0},
    "Tier 4": {"size_pp": 12.0, "gap_pp": 20.0, "range_pp": 25.0},
}
lims = limits_by_tier.get(tier_name, limits_by_tier["Tier 3"])

dev = salt["dev_size_pp"]
gap = float(salt["metrics_extra"]["gap_val_train_pp"])
rng = float(salt["metrics_extra"]["range_pp"])
keys_per_split = kmap["split"].value_counts().to_dict()

summary = pd.DataFrame([
    {"Métrica": "Desvío tamaño TRAIN (pp)", "Valor": dev["train"], "Límite": lims["size_pp"], "OK": dev["train"] <= lims["size_pp"]},
    {"Métrica": "Desvío tamaño VAL (pp)",   "Valor": dev["val"],   "Límite": lims["size_pp"], "OK": dev["val"]   <= lims["size_pp"]},
    {"Métrica": "Desvío tamaño TEST (pp)",  "Valor": dev["test"],  "Límite": lims["size_pp"], "OK": dev["test"]  <= lims["size_pp"]},
    {"Métrica": "Gap VAL–TRAIN (pp)",       "Valor": gap,          "Límite": lims["gap_pp"],  "OK": gap         <= lims["gap_pp"]},
    {"Métrica": "Rango prevalencias (pp)",  "Valor": rng,          "Límite": lims["range_pp"],"OK": rng         <= lims["range_pp"]},
])

# Guardar reporte para la memoria
qa_dir = PREP / "reports"; qa_dir.mkdir(exist_ok=True)
summary.to_csv(qa_dir / "split_checks.csv", index=False)

md_lines = [
    "# QA del split",
    f"- **Tier**: {tier_name} | **SALT**: {salt['salt']} | **bins**: {meta['bins']}",
    f"- **Ratios objetivo**: {meta['ratios_target']}",
    f"- **Claves únicas**: {len(kmap):,} | **Keys por split**: {keys_per_split}",
    "",
    "## Checks"
]
for _, r in summary.iterrows():
    md_lines.append(f"- {r['Métrica']}: {r['Valor']:.3f} (límite {r['Límite']}) → {'OK' if r['OK'] else 'NO'}")
(PREP / "reports" / "split_checks.md").write_text("\n".join(md_lines))

# Mostrar en el notebook
display(Markdown("## Validación del split"))
display(summary)
print("Reporte guardado en:", qa_dir)


## Validación del split

,Métrica,Valor,Límite,OK
0,Desvío tamaño TRAIN (pp),9.893983,10.0,True
1,Desvío tamaño VAL (pp),2.204555,10.0,True
2,Desvío tamaño TEST (pp),7.689428,10.0,True
3,Gap VAL–TRAIN (pp),7.449193,16.0,True
4,Rango prevalencias (pp),19.487988,20.0,True


Reporte guardado en: /content/drive/MyDrive/TFG/DATASETS/IDS2018/prepared/reports


In [ ]:
# ================================================================
# CELDA 3 — Aplicar split y exportar train/val/test (DEF)
#   · Join por clave -> split_code (0/1/2) usando keys_split_map.parquet
#   · Sin fuga: misma clave => mismo split
#   · Salida: PREP/splits/{train,val,test}/part_XXXXX.parquet
#   · Meta : PREP/splits/_SPLIT_ROWS_META.json
#   · Fast-path: IDS_REUSE_SPLITS=1 (por defecto) para reutilizar si ya existen
#   · Overwrite: IDS_OVERWRITE_SPLITS=1 para limpiar y rehacer
# ================================================================
import os, json, gc, time
from pathlib import Path
import numpy as np
import pandas as pd

def ts(): return time.strftime("%H:%M:%S")
def log(msg): print(f"[Celda3] {ts()} {msg}", flush=True)

# ---------- Precondiciones ----------
assert 'BASE' in globals() and 'PREP' in globals(), "Ejecuta antes Celdas 0–2."
BASE = Path(BASE); PREP = Path(PREP)
if 'PARQUET_DIR' not in globals(): PARQUET_DIR = PREP / "parquet_parts"
if 'MANIFESTS'   not in globals(): MANIFESTS   = PREP / "manifests"
PARQUET_DIR.mkdir(parents=True, exist_ok=True)

# ---------- Columnas clave y label ----------
diag_path = PREP / "_DIAG_DATA_COLS.json"
KEY_COLS = ['srcip','sport','dstip','dsport','proto']
LABEL_COL = 'label'
if diag_path.exists():
    try:
        diag = json.loads(diag_path.read_text())
        if isinstance(diag.get('KEY_COLS'), list) and diag['KEY_COLS']:
            KEY_COLS = [c.lower() for c in diag['KEY_COLS']]
        elif isinstance(diag.get('df_keys_cols'), list) and diag['df_keys_cols']:
            KEY_COLS = [c.lower() for c in diag['df_keys_cols']]
        LABEL_COL = (diag.get('LABEL_COL') or LABEL_COL)
    except Exception:
        pass
KEY_COLS = [c.lower() for c in KEY_COLS]
log(f"KEY_COLS={KEY_COLS} | LABEL_COL={LABEL_COL}")

# ---------- Mapa clave→split ----------
kmap_path = PREP / "keys_split_map.parquet"
assert kmap_path.exists(), f"No existe {kmap_path}; ejecuta Celda 2."
kmap = pd.read_parquet(kmap_path)  # columnas: key, split_code, split
kmap['key'] = kmap['key'].astype(str)
CODE2NAME = {0: "train", 1: "val", 2: "test"}

# ---------- Fast-path: reutilizar si ya hay splits ----------
SPLITS_DIR = PREP / "splits"
META_PATH  = SPLITS_DIR / "_SPLIT_ROWS_META.json"
REUSE_SPLITS     = bool(int(os.getenv("IDS_REUSE_SPLITS", "1")))   # 1=reusar
OVERWRITE_SPLITS = bool(int(os.getenv("IDS_OVERWRITE_SPLITS", "0")))# 1=limpiar y rehacer

if REUSE_SPLITS and META_PATH.exists() and (SPLITS_DIR/"train").exists() and (SPLITS_DIR/"val").exists() and (SPLITS_DIR/"test").exists():
    meta_rows = json.loads(META_PATH.read_text())
    log("♻️ Reutilizando splits existentes")
    for s in ("train","val","test"):
        m = meta_rows["by_split"][s]
        log(f"  {s:>5}: {m['rows']:,} filas | prev={m['prev']*100:.3f}% | files={m['files']}")
else:
    # ---------- Descubrir partes de datos ----------
    try:
        import pyarrow.parquet as pq
    except Exception:
        pq = None

    def _has_required_columns(p, required):
        if pq is None:
            try:
                pd.read_parquet(p, columns=list(required), engine="pyarrow"); return True
            except Exception: return False
        try:
            schema = pq.read_schema(str(p))
            cols = {f.name.lower() for f in schema}
            return set(map(str.lower, required)).issubset(cols)
        except Exception:
            return False

    def discover_parts_for_rows():
        required = set([*KEY_COLS, LABEL_COL])
        parts = []
        man = MANIFESTS / "prepared_manifest.json"
        if man.exists():
            try:
                j = json.loads(man.read_text())
                base = Path(j.get("parquet_dir", str(PARQUET_DIR)))
                for it in j.get("parts", []):
                    p = (base / it["part"]).resolve()
                    if p.is_file() and p.suffix == ".parquet" and _has_required_columns(p, required):
                        parts.append(p)
            except Exception:
                pass
        if not parts and PARQUET_DIR.exists():
            for p in sorted(PARQUET_DIR.glob("*.parquet")):
                name_low = p.name.lower()
                if any(k in name_low for k in ("keys_split_map","split_meta","salt_pick","features","scaler","idx_seq")):
                    continue
                if _has_required_columns(p, required):
                    parts.append(p)
        if not parts:
            for p in sorted(PREP.glob("*.parquet")):
                name_low = p.name.lower()
                if any(k in name_low for k in ("keys_split_map","split_meta","salt_pick","features","scaler","idx_seq")):
                    continue
                if _has_required_columns(p, required):
                    parts.append(p)
        return parts

    parts = discover_parts_for_rows()
    assert parts, "No hay partes válidas en PARQUET_DIR/manifest. Revisa Celda 1.5."
    log(f"Partes detectadas: {len(parts)}")

    # ---------- Carpetas de salida ----------
    OUT_TRAIN = SPLITS_DIR / "train"; OUT_VAL = SPLITS_DIR / "val"; OUT_TEST = SPLITS_DIR / "test"
    for d in (OUT_TRAIN, OUT_VAL, OUT_TEST): d.mkdir(parents=True, exist_ok=True)

    # Limpieza si se solicita overwrite
    if OVERWRITE_SPLITS:
        for d in (OUT_TRAIN, OUT_VAL, OUT_TEST):
            for fp in d.glob("*.parquet"):
                try: fp.unlink()
                except Exception: pass

    # ---------- Utilidades ----------
    def _make_key_frame(df):
        # normaliza columnas y compone la clave string (soporta flow_key o 5-tupla)
        df.columns = df.columns.str.lower()
        miss = [c for c in KEY_COLS+[LABEL_COL] if c not in df.columns]
        if miss: raise AssertionError(f"Faltan columnas: {miss}")
        key_str = df[KEY_COLS].astype(str).agg('|'.join, axis=1)
        y = pd.to_numeric(df[LABEL_COL], errors='coerce').fillna(0).astype(np.int8)
        return pd.DataFrame({"__key": key_str, LABEL_COL: y})

    def _write_part(df_full_with_split, out_dir: Path, idx: int):
        out_path = out_dir / f"part_{idx:05d}.parquet"
        df_full_with_split.to_parquet(out_path, index=False)

    # ---------- Procesar partes ----------
    meta_counts = { "train":{"rows":0,"pos":0,"neg":0},
                    "val":  {"rows":0,"pos":0,"neg":0},
                    "test": {"rows":0,"pos":0,"neg":0} }
    file_idx = {"train":0, "val":0, "test":0}
    unmatched_total = 0

    # Prepara kmap para el join
    kmap_join = kmap.rename(columns={"key":"__key"})[["__key","split_code","split"]]

    for i, p in enumerate(parts, 1):
        log(f"Parte {i}/{len(parts)} → {p.name}")
        df = pd.read_parquet(p, engine="pyarrow")          # leemos todo (queremos conservar todas las columnas)
        keydf = _make_key_frame(df)                         # crea __key y label limpio
        tmp = keydf.merge(kmap_join, on="__key", how="left")

        missing = int(tmp["split"].isna().sum())
        if missing:
            unmatched_total += missing
            keep_mask = ~tmp["split"].isna()
            df  = df.loc[keep_mask].reset_index(drop=True)
            tmp = tmp.loc[keep_mask].reset_index(drop=True)

        # añade split al df completo
        df["split_code"] = tmp["split_code"].astype(np.int8).values
        df["split"]      = tmp["split"].astype(str).values

        # escribe por split y acumula métricas
        for split_name, out_dir in (("train", OUT_TRAIN), ("val", OUT_VAL), ("test", OUT_TEST)):
            sub = df[df["split"] == split_name]
            if sub.empty: continue
            _write_part(sub, out_dir, file_idx[split_name]); file_idx[split_name] += 1

            y = pd.to_numeric(sub[LABEL_COL], errors="coerce").fillna(0).astype(np.int8).to_numpy()
            pos = int((y == 1).sum()); neg = int((y == 0).sum())
            meta_counts[split_name]["rows"] += len(sub)
            meta_counts[split_name]["pos"]  += pos
            meta_counts[split_name]["neg"]  += neg

        del df, keydf, tmp; gc.collect()

    # ---------- Resumen y meta ----------
    def _prev(pos, neg): d = pos+neg; return (pos/d) if d else 0.0

    tot_rows = sum(meta_counts[s]["rows"] for s in meta_counts)
    assert tot_rows > 0, "No se han escrito filas en los splits."

    meta_rows = {
        "rows_total": int(tot_rows),
        "unmatched_rows_dropped": int(unmatched_total),
        "by_split": {
            s: {
                "rows": int(meta_counts[s]["rows"]),
                "pos":  int(meta_counts[s]["pos"]),
                "neg":  int(meta_counts[s]["neg"]),
                "prev": float(_prev(meta_counts[s]["pos"], meta_counts[s]["neg"])),
                "files": int(file_idx[s]),
            } for s in ("train","val","test")
        }
    }
    SPLITS_DIR.mkdir(parents=True, exist_ok=True)
    META_PATH.write_text(json.dumps(meta_rows, indent=2))

    log("✔️ Exportado dataset particionado en PREP/splits/{train,val,test}")
    for s in ("train","val","test"):
        m = meta_rows["by_split"][s]
        log(f"  {s:>5}: {m['rows']:,} filas | prev={m['prev']*100:.3f}% | files={m['files']}")
    if unmatched_total:
        log(f"⚠️ Aviso: se descartaron {unmatched_total:,} filas cuyo key no estaba en keys_split_map.parquet (revisa manifest/Celda 2 si no esperabas esto).")

[Celda3] 21:39:17 KEY_COLS=['flow_key'] | LABEL_COL=label
[Celda3] 21:39:17 ♻️ Reutilizando splits existentes
[Celda3] 21:39:17   train: 1,890,770 filas | prev=27.206% | files=18
[Celda3] 21:39:17     val: 541,208 filas | prev=34.656% | files=18
[Celda3] 21:39:17    test: 713,747 filas | prev=46.694% | files=18


In [ ]:
# ============================================
# CELDA 4 (HARD-PRUNE) — QUEDARSE SÓLO CON TOP-25, RELABEL A 'y', ESCALAR CON TRAIN
# ============================================
from __future__ import annotations
from pathlib import Path
from datetime import datetime
import json, numpy as np, pandas as pd

def clog(m): print(f"[Celda4] {datetime.now().strftime('%H:%M:%S')} {m}", flush=True)

assert 'BASE' in globals(), "BASE no definido (ejecuta Celda 0)."
BASE = Path(BASE)
PREP = BASE/"prepared"
SPLITS = PREP/"splits"
FEAT   = PREP/"features"
for s in ("train","val","test"): (FEAT/s).mkdir(parents=True, exist_ok=True)

# --- TOP-25 (deploy-safe) BLOQUEADAS
TOP25 = [
    "fwd_seg_size_min","psh_flag_cnt","ack_flag_cnt","fwd_pkt_len_mean","fwd_act_data_pkts",
    "fwd_iat_mean","bwd_iat_tot","idle_mean","bwd_pkts_s","pkt_len_min",
    "totlen_fwd_pkts","urg_flag_cnt","bwd_iat_std","init_fwd_win_byts","bwd_iat_mean",
    "idle_min","flow_pkts_s","fwd_psh_flags","totlen_bwd_pkts","init_bwd_win_byts",
    "fin_flag_cnt","active_mean","active_std","pkt_len_var","flow_byts_s",
]
EXCLUDE = {  # por si aparecen: nunca se usan
    "srcip","dstip","source_ip","dest_ip","destination_ip",
    "sport","dsport","srcport","dstport","src_port","dst_port",
    "source_port","destination_port","proto","protocol",
    "flow_id","flow_key","split","split_code","label"  # (label se relabela a 'y')
}

def parts(split):
    d = SPLITS/split
    ps = sorted(d.glob("part_*.parquet")) or sorted(d.glob("*.parquet"))
    assert ps, f"No hay parts en {d}"
    return ps

# --- Detectar nombre real de la etiqueta en TRAIN y columnas disponibles
clog("Detectando etiqueta y columnas en TRAIN…")
df0 = pd.read_parquet(parts("train")[0])
cols_lower = {c.lower(): c for c in df0.columns}
LABEL_SRC = None
for cand in ("y","label","class","target"):
    if cand in cols_lower: LABEL_SRC = cols_lower[cand]; break
assert LABEL_SRC is not None, f"No encuentro columna de etiqueta en TRAIN (probé y/label/class/target). Columnas: {list(df0.columns)[:20]}"
AVAILABLE = set(df0.columns)
FEATURES = [c for c in TOP25 if c in AVAILABLE and c not in EXCLUDE]
MISSING  = [c for c in TOP25 if c not in FEATURES]
assert len(FEATURES) >= 12, f"Demasiado pocas TOP-25 presentes en TRAIN ({len(FEATURES)})."

clog(f"TOP-25 presentes: {len(FEATURES)} | faltan: {len(MISSING)} → se crearán como NaN antes de imputar.")

# --- Aprender scaler SOLO con TRAIN
clog("Aprendiendo Min–Max + medias en TRAIN…")
mins, maxs, sum_, cnt_ = {c: np.inf for c in TOP25}, {c: -np.inf for c in TOP25}, {c:0.0 for c in TOP25}, {c:0 for c in TOP25}

for p in parts("train"):
    df = pd.read_parquet(p)
    for m in MISSING:
        if m not in df.columns: df[m] = np.nan
    X = df[FEATURES].apply(pd.to_numeric, errors="coerce")
    mins_part = X.min(skipna=True); maxs_part = X.max(skipna=True)
    sum_part  = X.sum(skipna=True); cnt_part  = X.notna().sum()
    for c in FEATURES:
        vmin, vmax = float(mins_part[c]), float(maxs_part[c])
        mins[c] = min(mins[c], vmin if np.isfinite(vmin) else mins[c])
        maxs[c] = max(maxs[c], vmax if np.isfinite(vmax) else maxs[c])
        sum_[c] += float(sum_part[c]); cnt_[c] += int(cnt_part[c])

means = {c: (sum_[c]/cnt_[c] if cnt_[c]>0 else 0.0) for c in TOP25}
# Denominadores y saneo (constantes → den=1)
den = {}
for c in TOP25:
    mn = float(mins.get(c,0.0)); mx = float(maxs.get(c,mn))
    d = mx - mn
    if not np.isfinite(mn): mn = 0.0
    if not np.isfinite(mx): mx = mn
    if d <= 0.0: d = 1.0  # columna constante
    mins[c], maxs[c], den[c] = mn, mx, d

# --- Transformar/escribir sólo TOP-25 (+ y)
def transform_split(split):
    out = FEAT/split
    # limpia anteriores
    for old in out.glob("*.parquet"): old.unlink()
    nrows = 0
    for i,p in enumerate(parts(split)):
        df = pd.read_parquet(p)
        # quedarnos sólo con TOP-25 → crear las que falten
        for m in MISSING:
            if m not in df.columns: df[m] = np.nan
        X = df[FEATURES].apply(pd.to_numeric, errors="coerce").astype("float64")
        # imputar medias de TRAIN
        for c in FEATURES:
            X[c] = X[c].fillna(means[c])
            X[c] = (X[c] - mins[c]) / den[c]
        X = X.clip(0.0,1.0).astype("float32")
        # etiqueta a 'y'
        y = pd.to_numeric(df[LABEL_SRC], errors="coerce").fillna(0).astype("int8")
        out_df = pd.concat([X, y.rename("y")], axis=1)
        out_df.to_parquet(out/f"part_{i:05d}.parquet", index=False)
        nrows += len(out_df)
    return nrows

rows = {}
for s in ("train","val","test"):
    clog(f"Transformando {s} (TOP-25 hard-prune)…")
    rows[s] = transform_split(s)
    clog(f"  {s}: {rows[s]} filas")

# --- Guardar meta y scaler
ts = datetime.now().isoformat()
scaler_json = {
    "computed_on": "train","timestamp": ts,
    "features": FEATURES,  # las que realmente existen (subset de TOP25)
    "mins": {c: mins[c] for c in FEATURES},
    "maxs": {c: maxs[c] for c in FEATURES},
    "means": {c: means[c] for c in FEATURES},
    "note": "HARD-PRUNE Top-25; MinMax + imputación media (TRAIN); clip [0,1]; dtype float32."
}
(FEAT/"scaler_minmax.json").write_text(json.dumps(scaler_json, indent=2))

meta_json = {
    "features": FEATURES,
    "dtype": "float32",
    "label_col": "y",
    "scaler": "scaler_minmax.json",
    "rows": rows,
    "detect": {"whitelist":"top25_locked","no_5tuple":True,"computed":ts}
}
(FEAT/"features_meta.json").write_text(json.dumps(meta_json, indent=2))
clog(f"✅ HARD-PRUNE listo. Features={len(FEATURES)} (subset de TOP-25), escritos en prepared/features/*")


[Celda4] 21:39:21 Detectando etiqueta y columnas en TRAIN…
[Celda4] 21:39:24 TOP-25 presentes: 25 | faltan: 0 → se crearán como NaN antes de imputar.
[Celda4] 21:39:24 Aprendiendo Min–Max + medias en TRAIN…
[Celda4] 21:40:42 Transformando train (TOP-25 hard-prune)…
[Celda4] 21:41:36   train: 1890770 filas
[Celda4] 21:41:36 Transformando val (TOP-25 hard-prune)…
[Celda4] 21:42:31   val: 541208 filas
[Celda4] 21:42:31 Transformando test (TOP-25 hard-prune)…
[Celda4] 21:43:10   test: 713747 filas
[Celda4] 21:43:11 ✅ HARD-PRUNE listo. Features=25 (subset de TOP-25), escritos en prepared/features/*


In [ ]:
from pathlib import Path, PurePath
import pandas as pd
parts = sorted((BASE/"prepared"/"features"/"train").glob("part_*.parquet"))[:1]
df0 = pd.read_parquet(parts[0])
print("Cols:", list(df0.columns)[:10], "... total:", len(df0.columns))
print("y unique:", df0["y"].unique()[:10])


Cols: ['fwd_seg_size_min', 'psh_flag_cnt', 'ack_flag_cnt', 'fwd_pkt_len_mean', 'fwd_act_data_pkts', 'fwd_iat_mean', 'bwd_iat_tot', 'idle_mean', 'bwd_pkts_s', 'pkt_len_min'] ... total: 26
y unique: [0 1]


In [ ]:
# ============================================
# CELDA 4-POST — Setup tras Celda 4 (ENV + sanity + limpiar modelos)
#  · Deja los ENV *finales* (los que dieron las mejores métricas)
#  · Verifica features bloqueadas
#  · Limpia MODELS_DIR para no mezclar artefactos
# ============================================
from pathlib import Path
import os, json, shutil

assert 'BASE' in globals(), "BASE no definido (ejecuta Celda 0 y luego Celda 4)."
BASE = Path(BASE)
FEAT_DIR  = BASE / "prepared" / "features"
MODEL_DIR = BASE / "models_ids2018"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# --- 1) Leer meta de Celda 4 y sanity de features (TOP-25 whitelisted)
with open(FEAT_DIR/"features_meta.json","r") as f:
    meta = json.load(f)
FEATURES = meta["features"]
print(f"[4-POST] Features en meta (deberían ser 25): {len(FEATURES)}")
print("[4-POST] Preview:", FEATURES[:10], "..." if len(FEATURES)>10 else "")

PROHIBIDAS = {"label","y","srcip","dstip","sport","dsport","src_port","dst_port","proto","protocol","flow_id","flow_key"}
assert not (set(FEATURES) & PROHIBIDAS), f"Se coló alguna prohibida: {set(FEATURES)&PROHIBIDAS}"

# --- 2) Variables de entorno FINALES (las que quieres para reproducir resultados)
# Objetivo Neyman–Pearson (FPR 0.5% en VALIDATION) + muestra rápida de 600k
os.environ["IDS_MAX_FPR"]   = "0.005"   # 0.5% target FPR en VAL
os.environ["IDS_MIN_TPR"]   = "0.60"    # TPR mínimo deseado en VAL (para backoff)
os.environ["IDS_NP_MARGIN"] = "0.80"    # margen (lo usamos para reporting/consistencia)
os.environ["IDS_FAST"]      = "1"       # modo rápido ON
os.environ["IDS_FAST_ROWS"] = "600000"  # muestra rápida de TRAIN

# Extra (determinismo razonable)
os.environ["OMP_NUM_THREADS"] = os.environ.get("OMP_NUM_THREADS", "1")

print("[4-POST] ENV →",
      "IDS_MAX_FPR=", os.environ["IDS_MAX_FPR"],
      "| IDS_MIN_TPR=", os.environ["IDS_MIN_TPR"],
      "| IDS_NP_MARGIN=", os.environ["IDS_NP_MARGIN"],
      "| IDS_FAST=", os.environ["IDS_FAST"],
      "| IDS_FAST_ROWS=", os.environ["IDS_FAST_ROWS"],
      "| OMP_NUM_THREADS=", os.environ["OMP_NUM_THREADS"])

# --- 3) Limpiar modelos previos
for f in MODEL_DIR.glob("*"):
    try:
        if f.is_file() or f.is_symlink():
            f.unlink()
        else:
            shutil.rmtree(f)
    except Exception as e:
        print("[4-POST] Aviso al limpiar:", f, e)

print("[4-POST] models_ids2018 limpio. Ya puedes ejecutar la Celda 5 (DEF).")


[4-POST] Features en meta (deberían ser 25): 25
[4-POST] Preview: ['fwd_seg_size_min', 'psh_flag_cnt', 'ack_flag_cnt', 'fwd_pkt_len_mean', 'fwd_act_data_pkts', 'fwd_iat_mean', 'bwd_iat_tot', 'idle_mean', 'bwd_pkts_s', 'pkt_len_min'] ...
[4-POST] ENV → IDS_MAX_FPR= 0.005 | IDS_MIN_TPR= 0.60 | IDS_NP_MARGIN= 0.80 | IDS_FAST= 1 | IDS_FAST_ROWS= 600000 | OMP_NUM_THREADS= 1
[4-POST] models_ids2018 limpio. Ya puedes ejecutar la Celda 5 (DEF).


In [ ]:
# ============================================
# CELDA 5 (DEF) — HGB + τ Neyman–Pearson (FPR 0.5%) + FAST_ROWS=600k
#  · Lee los ENV fijados en 4-POST (no los vuelve a escribir)
#  · Entrena HGB robusto
#  · Selecciona τ en VALIDATION con NP y evalúa en TEST
#  · Guarda artefactos
# ============================================
from __future__ import annotations
from pathlib import Path
import os, json, warnings
import numpy as np
import pandas as pd
import joblib
import time

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import (
    roc_curve, roc_auc_score, average_precision_score, confusion_matrix,
    precision_score, recall_score, f1_score
)

warnings.filterwarnings("ignore", category=UserWarning)
def ts(): import time; return time.strftime("%H:%M:%S")
def log(m): print(f"[Celda5] {ts()} {m}", flush=True)

# --------- Rutas y meta de features ---------
assert 'BASE' in globals(), "BASE no definido (ejecuta Celdas 0–4 y 4-POST)."
BASE = Path(BASE)
FEAT_DIR  = BASE/"prepared"/"features"
MODEL_DIR = BASE/"models_ids2018"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

with open(FEAT_DIR/"features_meta.json","r") as f:
    meta = json.load(f)
FEATURES = meta["features"]
label_col = meta.get("label_col","y")

# --------- Chequeos anti-fuga / integridad ---------
assert len(FEATURES) == 25, f"Esperaba 25 features whitelisted; tengo {len(FEATURES)}"
_prohibidas = {"label","y","srcip","dstip","sport","dsport","src_port","dst_port","proto","protocol","flow_id","flow_key","split","split_code"}
assert not (_prohibidas & set(FEATURES)), f"Prohibidas colándose en FEATURES: {_prohibidas & set(FEATURES)}"
log("WHITELIST LOCKED → 25 columnas")
log(f"Preview: {FEATURES[:10]} ...")

# --------- ENV (usamos lo fijado en 4-POST; snapshot de seguridad) ---------
MAX_FPR   = float(os.getenv("IDS_MAX_FPR", "0.005"))
MIN_TPR   = float(os.getenv("IDS_MIN_TPR", "0.60"))
NP_MARGIN = float(os.getenv("IDS_NP_MARGIN", "0.80"))
FAST      = bool(int(os.getenv("IDS_FAST", "1")))
FAST_ROWS = int(os.getenv("IDS_FAST_ROWS", "600000"))
log(f"ENV usados → MAX_FPR={MAX_FPR} | MIN_TPR={MIN_TPR} | NP_MARGIN={NP_MARGIN} | FAST={FAST} | FAST_ROWS={FAST_ROWS}")

# --------- Utilidades de carga ---------
def _parts(split:str):
    d = FEAT_DIR/split
    ps = sorted(d.glob("part_*.parquet")) or sorted(d.glob("*.parquet"))
    assert ps, f"No hay parts en {d}"
    return ps

def load_split(split:str, sample_n:int|None=None):
    log(f"Cargando {split.upper()}…")
    dfs = []
    for p in _parts(split):
        df = pd.read_parquet(p, columns=[*FEATURES, label_col])
        dfs.append(df)
    df = pd.concat(dfs, ignore_index=True)
    y = df[label_col].astype(np.int8).to_numpy()
    X = df[FEATURES].astype("float32")
    prev = float((y == 1).mean())
    log(f"  {split.upper():5s}: X=({len(df):,}, {X.shape[1]}) | prev={prev:.4f}")

    if sample_n is not None and sample_n < len(df):
        n_pos = int(y.sum()); n_neg = len(y) - n_pos
        target_pos = int(round(sample_n * (n_pos/len(y))))
        target_neg = sample_n - target_pos
        pos_idx = np.flatnonzero(y == 1)
        neg_idx = np.flatnonzero(y == 0)
        rng = np.random.default_rng(42)
        take_pos = rng.choice(pos_idx, size=min(target_pos, len(pos_idx)), replace=False)
        take_neg = rng.choice(neg_idx, size=min(target_neg, len(neg_idx)), replace=False)
        take = np.concatenate([take_pos, take_neg]); rng.shuffle(take)
        X = X.iloc[take].reset_index(drop=True); y = y[take]
        log(f"  MODO RÁPIDO → muestreo de {len(y):,}")
    return X, y, prev

# --------- Cargar datos ---------
X_train, y_train, prev_train = load_split("train", sample_n=FAST_ROWS if FAST else None)
X_val,   y_val,   prev_val   = load_split("val",   sample_n=None)

# --------- Class weights (inversos a prevalencia; media≈1) ---------
w_pos = 1.0 / max(1e-9, 2.0*prev_train)
w_neg = 1.0 / max(1e-9, 2.0*(1.0 - prev_train))
log(f"Pesos → w_neg={w_neg:.3f} | w_pos={w_pos:.3f}")

# --------- Entrenamiento HGB (robusto y un poco más capaz) ---------
hgb_params = dict(
    max_iter=300,
    learning_rate=0.10,
    max_depth=None,
    min_samples_leaf=250,
    l2_regularization=0.0,
    early_stopping=True,
    validation_fraction=0.12,
    n_iter_no_change=30,
    random_state=42,
    class_weight={0: w_neg, 1: w_pos}
)
log("Entrenando HGB…")
hgb = HistGradientBoostingClassifier(**hgb_params)
hgb.fit(X_train, y_train)
# --- cronometrar fit + iteraciones y snapshot
t0 = time.time()
hgb.fit(X_train, y_train)
fit_time_sec = time.time() - t0

n_iter_used = int(getattr(hgb, "n_iter_", 0))
bench_partial = {
    "fit_time_sec": float(fit_time_sec),
    "n_iter_used": n_iter_used
}
(Path(MODEL_DIR/"_bench_fit_partial.json")).write_text(json.dumps(bench_partial, indent=2))
# ------------------------------------------------------------------------

log("Entrenamiento completo")

# --------- Probabilidades en VAL y percentiles ---------
val_prob = hgb.predict_proba(X_val)[:,1]
pcts = np.percentile(val_prob, [0,1,5,50,90,99,99.5,99.9,99.99,100]).tolist()
log(f"VAL prob percentiles: {np.array2string(np.array(pcts), precision=6, suppress_small=True)}")

# --------- Selección de τ (Neyman–Pearson) ---------
def pick_threshold_np(y_true, prob, max_fpr=0.005, min_tpr=0.60):
    fpr, tpr, thr = roc_curve(y_true, prob)
    mask_ok = fpr <= max_fpr + 1e-12
    if not np.any(mask_ok):
        idx = int(np.argmin(np.maximum(fpr - max_fpr, 0.0)))
        return float(thr[idx]), float(fpr[idx]), float(tpr[idx])
    idxs = np.where(mask_ok)[0]
    best = max(idxs, key=lambda i: (tpr[i], thr[i]))
    if tpr[best] < min_tpr:
        best_f1, best_i = -1.0, best
        for i in idxs:
            yhat = (prob >= thr[i]).astype(int)
            _f1 = f1_score(y_true, yhat, zero_division=0)
            if (_f1 > best_f1) or (np.isclose(_f1, best_f1) and thr[i] > thr[best_i]):
                best_f1, best_i = _f1, i
        best = best_i
    return float(thr[best]), float(fpr[best]), float(tpr[best])

tau_np, fpr_val, tpr_val = pick_threshold_np(y_val, val_prob, MAX_FPR, MIN_TPR)

# --------- Métricas helper ---------
def summarize_metrics(y, prob, thr):
    yhat = (prob >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y, yhat, labels=[0,1]).ravel()
    return {
        "AUROC": float(roc_auc_score(y, prob)),
        "AUPRC": float(average_precision_score(y, prob)),
        "Precision": float(precision_score(y, yhat, zero_division=0)),
        "Recall": float(recall_score(y, yhat, zero_division=0)),
        "F1": float(f1_score(y, yhat, zero_division=0)),
        "FPR": float(fp / max(1, (fp+tn))),
        "TPR": float(tp / max(1, (tp+fn))),
        "threshold_used": float(thr),
        "confusion": {"tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)},
    }

metrics_val = summarize_metrics(y_val, val_prob, tau_np)
log(f"NP VAL → thr={tau_np:.6f} | FPR={metrics_val['FPR']:.4f} | TPR={metrics_val['TPR']:.4f} | F1={metrics_val['F1']:.4f}")
(Path(MODEL_DIR/"metrics_val.json")).write_text(json.dumps(metrics_val, indent=2))
log("✔️ metrics_val.json escrito")

# --------- Cargar TEST y evaluar con el mismo τ ---------
log("Cargando TEST…")
X_test_parts, y_test_parts = [], []
for p in _parts("test"):
    df = pd.read_parquet(p, columns=[*FEATURES, label_col])
    X_test_parts.append(df[FEATURES].astype("float32"))
    y_test_parts.append(df[label_col].astype(np.int8).to_numpy())
X_test = pd.concat(X_test_parts, ignore_index=True)
y_test  = np.concatenate(y_test_parts)

test_prob = hgb.predict_proba(X_test)[:,1]
pcts_test = np.percentile(test_prob, [0,1,5,50,90,99,99.5,99.9,99.99,100]).tolist()
log(f"TEST prob percentiles: {np.array2string(np.array(pcts_test), precision=6, suppress_small=True)}")

metrics_test = summarize_metrics(y_test, test_prob, tau_np)
log("✔️ Evaluación en TEST")
log(f"  AUROC={roc_auc_score(y_test, test_prob):.4f} | AUPRC={average_precision_score(y_test, test_prob):.4f}")
log(f"  Prec={metrics_test['Precision']:.4f} | Rec={metrics_test['Recall']:.4f} | F1={metrics_test['F1']:.4f}")
log(f"  CM: tn={metrics_test['confusion']['tn']} fp={metrics_test['confusion']['fp']} "
    f"fn={metrics_test['confusion']['fn']} tp={metrics_test['confusion']['tp']}")

# --------- Guardar artefactos ---------
joblib.dump(hgb, MODEL_DIR/"hgb_model.pkl")
(Path(MODEL_DIR/"hgb_params.json")).write_text(json.dumps(hgb.get_params(), indent=2))

decision = {
    "selected": {
        "name": "np_strict",
        "tau": float(tau_np),
        "val_fpr": float(metrics_val["FPR"]),
        "val_tpr": float(metrics_val["TPR"]),
        "val_f1":  float(metrics_val["F1"]),
        "max_fpr_target": float(MAX_FPR),
        "min_tpr_target": float(MIN_TPR),
        "np_margin": float(NP_MARGIN),
    },
    "notes": "Threshold selected on VALIDATION with Neyman–Pearson (strict FPR); same τ applied to TEST."
}
(Path(MODEL_DIR/"decision_thresholds.json")).write_text(json.dumps(decision, indent=2))
(Path(MODEL_DIR/"metrics_test.json")).write_text(json.dumps(metrics_test, indent=2))

log("✔️ Artefactos guardados en MODELS_DIR")


[Celda5] 21:45:08 WHITELIST LOCKED → 25 columnas
[Celda5] 21:45:08 Preview: ['fwd_seg_size_min', 'psh_flag_cnt', 'ack_flag_cnt', 'fwd_pkt_len_mean', 'fwd_act_data_pkts', 'fwd_iat_mean', 'bwd_iat_tot', 'idle_mean', 'bwd_pkts_s', 'pkt_len_min'] ...
[Celda5] 21:45:08 ENV usados → MAX_FPR=0.005 | MIN_TPR=0.6 | NP_MARGIN=0.8 | FAST=True | FAST_ROWS=600000
[Celda5] 21:45:08 Cargando TRAIN…
[Celda5] 21:45:09   TRAIN: X=(1,890,770, 25) | prev=0.2721
[Celda5] 21:45:09   MODO RÁPIDO → muestreo de 600,000
[Celda5] 21:45:09 Cargando VAL…
[Celda5] 21:45:10   VAL  : X=(541,208, 25) | prev=0.3466
[Celda5] 21:45:10 Pesos → w_neg=0.687 | w_pos=1.838
[Celda5] 21:45:10 Entrenando HGB…
[Celda5] 21:46:06 Entrenamiento completo
[Celda5] 21:46:14 VAL prob percentiles: [0.       0.       0.       0.       0.000045 0.00005  0.000066 0.001526
 0.457339 0.999909]
[Celda5] 21:46:23 NP VAL → thr=0.000045 | FPR=0.0034 | TPR=0.5001 | F1=0.6639
[Celda5] 21:46:23 ✔️ metrics_val.json escrito
[Celda5] 21:46:23 Cargando 

In [ ]:
# ================================================================
# CELDA 5.8 — Corbes ROC i PR (VAL/TEST) amb punt τ* marcat
#   • Llegeix decision_thresholds/metrics i pinta ROC i PR per VAL/TEST
#   • Marca el punt operatiu τ* i anota AUROC/AUPRC als títols
# ================================================================
import os, json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    roc_curve, precision_recall_curve, roc_auc_score, average_precision_score
)
import joblib

BASE = Path(os.environ.get("IDS_BASE", "/content/drive/MyDrive/TFG/DATASETS/IDS2018"))
FEAT = BASE / "prepared" / "features"
MOD  = BASE / "models_ids2018"

# --- carregar features i model
meta = json.loads((FEAT/"features_meta.json").read_text())
FEATURES = meta["features"]
label_col = meta.get("label_col","y")
hgb = joblib.load(MOD/"hgb_model.pkl")

def load_xy(split: str):
    parts = sorted((FEAT/split).glob("part_*.parquet")) or sorted((FEAT/split).glob("*.parquet"))
    assert parts, f"No hi ha parts a {FEAT/split}"
    dfs = [pd.read_parquet(p, columns=[*FEATURES, label_col]) for p in parts]
    df = pd.concat(dfs, ignore_index=True)
    X = df[FEATURES].astype("float32").values
    y = df[label_col].astype("int8").values
    return X, y

# --- dades i scores
Xv, yv = load_xy("val")
Xt, yt = load_xy("test")
sv = hgb.predict_proba(Xv)[:,1]
st = hgb.predict_proba(Xt)[:,1]

# --- τ* i α (de la decisió NP)
dec = json.loads((MOD/"decision_thresholds.json").read_text())
tau = float(dec["selected"]["tau"])
alpha = float(dec["selected"].get("max_fpr_target", 0.005))
tau_star = r'$\tau^\ast$'  # etiqueta tipogràfica "τ*"

# ===================== ROC =====================
fpr_v, tpr_v, thr_v = roc_curve(yv, sv)
fpr_t, tpr_t, thr_t = roc_curve(yt, st)
auroc_v = roc_auc_score(yv, sv)
auroc_t = roc_auc_score(yt, st)

def point_at_tau(thr, x, y, tau):
    i = (np.abs(thr - tau)).argmin()
    return x[i], y[i]

px_v, py_v = point_at_tau(thr_v, fpr_v, tpr_v, tau)
px_t, py_t = point_at_tau(thr_t, fpr_t, tpr_t, tau)

plt.figure()
plt.plot(fpr_v, tpr_v, label=f"VAL (AUROC={auroc_v:.5f})")
plt.plot(fpr_t, tpr_t, label=f"TEST (AUROC={auroc_t:.5f})")
plt.scatter([px_v],[py_v], marker="o", s=35, label=f"{tau_star} (VAL)  FPR={px_v:.3f}")
plt.scatter([px_t],[py_t], marker="s", s=35, label=f"{tau_star} (TEST) FPR={px_t:.3f}")
plt.plot([0,1],[0,1], ls="--", lw=0.8, color="gray")
plt.xlim(0, 1); plt.ylim(0, 1)
plt.xlabel("FPR"); plt.ylabel("TPR")
plt.title(f"ROC — VAL i TEST (punt {tau_star} marcat)")
plt.legend(loc="lower right")
plt.grid(alpha=0.2); plt.tight_layout()
plt.savefig(MOD/"roc_val_test.png", dpi=300); plt.close()

# ===================== PR =====================
prec_v, rec_v, th_v = precision_recall_curve(yv, sv)
prec_t, rec_t, th_t = precision_recall_curve(yt, st)
auprc_v = average_precision_score(yv, sv)
auprc_t = average_precision_score(yt, st)

def pr_at_tau(thr, prec, rec, tau):
    i = (np.abs(thr - tau)).argmin()
    return prec[i], rec[i]

pp_v, rr_v = pr_at_tau(th_v, prec_v, rec_v, tau)
pp_t, rr_t = pr_at_tau(th_t, prec_t, rec_t, tau)

plt.figure()
plt.plot(rec_v, prec_v, label=f"VAL (AUPRC={auprc_v:.5f})")
plt.plot(rec_t, prec_t, label=f"TEST (AUPRC={auprc_t:.5f})")
plt.scatter([rr_v],[pp_v], marker="o", s=35, label=f"{tau_star} (VAL)")
plt.scatter([rr_t],[pp_t], marker="s", s=35, label=f"{tau_star} (TEST)")
plt.xlim(0, 1); plt.ylim(0, 1)
plt.xlabel("Recall"); plt.ylabel("Precision")
plt.title(f"Precision–Recall — VAL i TEST (punt {tau_star} marcat)")
plt.legend(loc="lower left")
plt.grid(alpha=0.2); plt.tight_layout()
plt.savefig(MOD/"pr_val_test.png", dpi=300); plt.close()

print("✔️ Figures guardades:",
      MOD/"roc_val_test.png",
      MOD/"pr_val_test.png")


✔️ Figures guardades: /content/drive/MyDrive/TFG/DATASETS/IDS2018/models_ids2018/roc_val_test.png /content/drive/MyDrive/TFG/DATASETS/IDS2018/models_ids2018/pr_val_test.png


In [ ]:
# ================================================================
# CELDA 6 — Cost computacional (CPU) i recursos → Taula 4.4
#   · Lee fit_time e iteraciones (Celda 5)
#   · Tamaño del modelo en disco
#   · Latencia y throughput de inferencia (batch real)
#   · Pico de memoria (aprox. RSS durante inferencia)
#   · Maquinari i Sistema/Python (de Celda 0)
#   · Emite JSON resumen y print “copiar/pegar” para la tabla 4.4
# ================================================================
from pathlib import Path
import os, json, time, platform, math
import joblib, psutil, numpy as np, pandas as pd

BASE = Path(os.environ.get("IDS_BASE", "/content/drive/MyDrive/TFG/DATASETS/IDS2018")).resolve()
FEAT_DIR  = BASE/"prepared"/"features"
MODEL_DIR = BASE/"models_ids2018"
PREP      = BASE/"prepared"

# --- 1) Cargar modelo y meta
hgb = joblib.load(MODEL_DIR/"hgb_model.pkl")
meta = json.loads((FEAT_DIR/"features_meta.json").read_text())
FEATURES  = meta["features"]; label_col = meta.get("label_col","y")

# --- 2) Fit time + n_iter (de Celda 5)
bench_partial_path = MODEL_DIR/"_bench_fit_partial.json"
if bench_partial_path.exists():
    _bp = json.loads(bench_partial_path.read_text())
    fit_time_sec = float(_bp.get("fit_time_sec", float("nan")))
    n_iter_used  = int(_bp.get("n_iter_used", 0))
else:
    fit_time_sec = float("nan"); n_iter_used = int(getattr(hgb, "n_iter_", 0))

# --- 3) Tamaño del modelo
model_size_mb = (MODEL_DIR/"hgb_model.pkl").stat().st_size / (1024*1024)

# --- 4) Latencia y throughput de inferencia (sobre TEST real; sin reescalar nada)
#     Usa un lote suficientemente grande para estabilizar (p.ej., 200k filas o todas si hay menos)
def _parts(split):
    d = FEAT_DIR/split
    ps = sorted(d.glob("part_*.parquet")) or sorted(d.glob("*.parquet"))
    assert ps, f"No hay parts en {d}"
    return ps

# Cargar TEST en bloques, cronometrando sólo predict_proba
target_rows = 200_000
rows_loaded, t_infer, rss_peak = 0, 0.0, 0
proc = psutil.Process(os.getpid())

for p in _parts("test"):
    df = pd.read_parquet(p, columns=[*FEATURES])
    if rows_loaded >= target_rows: break
    take = min(len(df), target_rows - rows_loaded)
    if take < len(df): df = df.iloc[:take]
    X = df[FEATURES].astype("float32")
    rss_peak = max(rss_peak, proc.memory_info().rss)
    t0 = time.time()
    _ = hgb.predict_proba(X)[:,1]
    t_infer += (time.time() - t0)
    rows_loaded += len(X)
    rss_peak = max(rss_peak, proc.memory_info().rss)

lat_ms_per_flow = (t_infer / max(1, rows_loaded)) * 1000.0
throughput_fps  = rows_loaded / t_infer if t_infer > 0 else float("nan")
peak_ram_gib    = rss_peak / (1024**3)

# --- 5) Sistema / Python (Celda 0 ya guardó versions y env)
VERS = json.loads((PREP/"versions.json").read_text())
ENV  = json.loads((PREP/"_ENV_IDS2018.json").read_text())
cpu_name = platform.processor() or platform.machine()
ram_total_gb = psutil.virtual_memory().total / (1024**3)
py_ver = VERS.get("python"); pd_ver = VERS.get("pandas"); np_ver = VERS.get("numpy")

# --- 6) Resumen para Taula 4.4
bench = {
    "Temps d’entrenament (fit) [min]": round(fit_time_sec/60.0, 2) if math.isfinite(fit_time_sec) else None,
    "Iteracions utilitzades (early stopping)": n_iter_used,
    "Pic de memòria (Peak RAM) [GiB]": round(peak_ram_gib, 2),
    "Mida del model (disc) [MB]": round(model_size_mb, 1),
    "Latència per flux (inferència) [ms/flux]": round(lat_ms_per_flow, 3),
    "Throughput estimat [flux/s]": int(round(throughput_fps)) if math.isfinite(throughput_fps) else None,
    "Maquinari (CPU / RAM)": f"{cpu_name} / {ram_total_gb:.1f} GB",
    "Sistema / Python": f"{VERS.get('platform','?')} / {py_ver}",
    "Pandas / NumPy / scikit-learn": f"{pd_ver} / {np_ver} / {VERS.get('sklearn')}"
}
(MODEL_DIR/"_bench_costs.json").write_text(json.dumps(bench, indent=2))
print("=== Taula 4.4 — valors recomanats ===")
for k,v in bench.items():
    print(f"{k:45s} {v}")
print("\nNota: Latència i throughput mesurats sobre TEST (lot≈200k), pipeline d’inferència estàndard.")


=== Taula 4.4 — valors recomanats ===
Temps d’entrenament (fit) [min]               0.47
Iteracions utilitzades (early stopping)       177
Pic de memòria (Peak RAM) [GiB]               1.0
Mida del model (disc) [MB]                    0.7
Latència per flux (inferència) [ms/flux]      0.016
Throughput estimat [flux/s]                   62969
Maquinari (CPU / RAM)                         x86_64 / 12.7 GB
Sistema / Python                              Linux-6.6.105+-x86_64-with-glibc2.35 / 3.12.12
Pandas / NumPy / scikit-learn                 2.2.2 / 2.0.2 / 1.6.1

Nota: Latència i throughput mesurats sobre TEST (lot≈200k), pipeline d’inferència estàndard.


In [ ]:
# ============================================
# CELDA QA (DEF) — Anti-fuga + Integridad + Consistencia τ + Sanity permutado
# Requisitos: Celdas 4, 4-POST y 5 ejecutadas
# ============================================
from __future__ import annotations
from pathlib import Path
import json, numpy as np, pandas as pd
from sklearn.metrics import (
    roc_auc_score, average_precision_score, confusion_matrix,
    precision_score, recall_score, f1_score
)
import joblib, math, warnings
warnings.filterwarnings("ignore")

def ts(): import time; return time.strftime("%H:%M:%S")
def log(m): print(f"[QA] {ts()} {m}", flush=True)

# --- Rutas básicas ---
assert 'BASE' in globals(), "Define BASE (ejecuta Celda 0)."
BASE = Path(BASE)
FEAT_DIR  = BASE/"prepared"/"features"
MODEL_DIR = BASE/"models_ids2018"

# --- 1) Cargar metadatos y chequear whitelist ---
meta = json.loads((FEAT_DIR/"features_meta.json").read_text())
FEATURES = list(meta["features"])
label_col = meta.get("label_col","y")
assert len(FEATURES) == 25, f"Esperaba 25 features; tengo {len(FEATURES)}"
PROHIBIDAS = {"label","y","srcip","dstip","sport","dsport","src_port","dst_port","proto","protocol","flow_id","flow_key","split","split_code"}
intr = set(FEATURES) & PROHIBIDAS
assert not intr, f"Prohibidas colándose en FEATURES: {intr}"
log(f"Meta OK → 25 features whitelisted: {FEATURES[:10]}...")

# --- Helpers de parts ---
def parts(split):
    d = FEAT_DIR/split
    ps = sorted(d.glob("part_*.parquet")) or sorted(d.glob("*.parquet"))
    assert ps, f"No hay parts en {d}"
    return ps

# --- 2) Integridad por split ---
def scan_split(split):
    mins = {c: +np.inf for c in FEATURES}
    maxs = {c: -np.inf for c in FEATURES}
    n_rows = 0; n_pos = 0; dtypes_ok = True; nnan = 0
    for p in parts(split):
        df = pd.read_parquet(p, engine="pyarrow")
        cols = list(df.columns)
        assert cols[-1] == label_col, f"'{label_col}' debe ir al final en {p.name}"
        assert set(cols[:-1]) == set(FEATURES), f"Columnas inesperadas en {p.name}"
        if not all(str(df[c].dtype) == "float32" for c in FEATURES): dtypes_ok = False
        if str(df[label_col].dtype) not in ("int8","int32","int64","uint8"): dtypes_ok = False
        X = df[FEATURES]
        nnan += int(X.isna().sum().sum())
        mi = X.min().to_dict(); ma = X.max().to_dict()
        for c in FEATURES:
            if mi[c] < mins[c]: mins[c] = float(mi[c])
            if ma[c] > maxs[c]: maxs[c] = float(ma[c])
        y = df[label_col].to_numpy()
        n_rows += len(y); n_pos += int((y == 1).sum())
    n_neg = n_rows - n_pos
    eps = 1e-6
    in_range = (min(mins.values()) >= -eps) and (max(maxs.values()) <= 1.0+eps)
    return {"rows": n_rows, "pos": n_pos, "neg": n_neg, "prev": n_pos/max(1,n_rows),
            "dtype_ok": dtypes_ok, "nnan": nnan, "min": min(mins.values()),
            "max": max(maxs.values()), "in_0_1": in_range}

summary = {}
for s in ("train","val","test"):
    m = scan_split(s); summary[s] = m
    log(f"{s.upper():5s}: rows={m['rows']:,} | prev={m['prev']*100:.3f}% | dtype_ok={m['dtype_ok']} | "
        f"NaNs={m['nnan']} | min={m['min']:.6f} | max={m['max']:.6f} | in[0,1]={m['in_0_1']}")

assert all(m["dtype_ok"] for m in summary.values()), "Tipos no conformes (esperado float32 en X, int en y)"
assert all(m["nnan"] == 0 for m in summary.values()), "Hay NaNs en features tras imputación"
assert all(m["in_0_1"] for m in summary.values()), "Algún feature fuera de [0,1]"

# --- 3) Sanity anti-fuga robusto: AUROC con y permutada ≈ 0.5 ---
# Usa muestreo proporcional aleatorio de TODO VAL y maneja scores degenerados.
def load_val_sample_rand(target=200_000, seed=123):
    sizes = [len(pd.read_parquet(p, columns=[label_col], engine="pyarrow")) for p in parts("val")]
    total = sum(sizes); rng = np.random.default_rng(seed)
    take = [int(round(target * (sz/total))) for sz in sizes]
    # Ajustar por redondeo
    diff = target - sum(take)
    for _ in range(abs(diff)):
        i = rng.integers(0, len(take))
        take[i] += 1 if diff > 0 else (-1 if take[i] > 0 else 0)
    Xs, ys = [], []
    for p, k in zip(parts("val"), take):
        if k <= 0: continue
        df = pd.read_parquet(p, columns=[*FEATURES, label_col], engine="pyarrow")
        idx = rng.choice(len(df), size=min(k, len(df)), replace=False)
        Xs.append(df.iloc[idx][FEATURES].to_numpy(dtype=np.float32))
        ys.append(df.iloc[idx][label_col].to_numpy())
    X = np.vstack(Xs); y = np.concatenate(ys)
    return X, y

hgb = joblib.load(MODEL_DIR/"hgb_model.pkl")
Xv, yv = load_val_sample_rand(200_000, seed=123)
pv = hgb.predict_proba(Xv)[:,1].astype(np.float64)

# Diagnóstico de degeneración
n_nan = int(np.isnan(pv).sum())
u = np.unique(pv[~np.isnan(pv)])
if n_nan > 0 or len(u) <= 1:
    auroc_perm = 0.5  # fallback: no hay capacidad de ranking (scores constantes/NaN)
    note = "DEGENERATE_SCORES_FALLBACK"
else:
    rng = np.random.default_rng(321)
    y_perm = rng.permutation(yv)
    try:
        auroc_perm = float(roc_auc_score(y_perm, pv))
        note = "OK"
    except Exception:
        auroc_perm = 0.5
        note = "EXCEPTION_FALLBACK"

log(f"Sanity permutado VAL → AUROC={auroc_perm:.4f} [{note}] (esperado ~0.5)")
assert 0.45 <= auroc_perm <= 0.55, "Sanity permutado fuera de [0.45,0.55] (revisa muestreo o posibles fugas)"

# --- 4) Consistencia τ y métricas (recalcular vs ficheros) ---
dec = json.loads((MODEL_DIR/"decision_thresholds.json").read_text())
tau = float(dec["selected"]["tau"])
log(f"τ seleccionado = {tau:.6f}")

def summarize(y, prob, thr):
    yhat = (prob >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y, yhat, labels=[0,1]).ravel()
    return {
        "Precision": float(precision_score(y, yhat, zero_division=0)),
        "Recall": float(recall_score(y, yhat, zero_division=0)),
        "F1": float(f1_score(y, yhat, zero_division=0)),
        "FPR": float(fp / max(1, fp+tn)),
        "TPR": float(tp / max(1, tp+fn)),
        "confusion": {"tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)},
        "AUROC": float(roc_auc_score(y, prob)),
        "AUPRC": float(average_precision_score(y, prob)),
    }

def recompute_split(split, batch=200_000):
    probs = []; labels = []
    for p in parts(split):
        df = pd.read_parquet(p, columns=[*FEATURES, label_col], engine="pyarrow")
        # batch para contener memoria
        for i in range(0, len(df), batch):
            Xb = df.iloc[i:i+batch][FEATURES].to_numpy(dtype=np.float32)
            probs.append(hgb.predict_proba(Xb)[:,1])
            labels.append(df.iloc[i:i+batch][label_col].to_numpy())
    prob = np.concatenate(probs); y = np.concatenate(labels)
    return summarize(y, prob, tau)

m_val_re = recompute_split("val")
m_tst_re = recompute_split("test")
m_val_file  = json.loads((MODEL_DIR/"metrics_val.json").read_text())
m_test_file = json.loads((MODEL_DIR/"metrics_test.json").read_text())

def _close(a,b,eps_abs=5e-4, eps_rel=1e-3):
    a, b = float(a), float(b)
    return abs(a-b) <= max(eps_abs, eps_rel*max(1.0, abs(b)))

assert _close(m_val_re["F1"], m_val_file["F1"]) and _close(m_val_re["TPR"], m_val_file["TPR"]) and _close(m_val_re["FPR"], m_val_file["FPR"]), \
    f"VAL mismatch vs metrics_val.json:\n  recomputed={m_val_re}\n  file={m_val_file}"
log("Consistencia VAL vs metrics_val.json → OK")

assert _close(m_tst_re["Precision"], m_test_file["Precision"]) and _close(m_tst_re["Recall"], m_test_file["Recall"]) and _close(m_tst_re["F1"], m_test_file["F1"]), \
    f"TEST mismatch vs metrics_test.json:\n  recomputed={m_tst_re}\n  file={m_test_file}"
log("Consistencia TEST vs metrics_test.json → OK")

log("✅ QA COMPLETA: sin fuga, integridad OK, sanity permutado ≈ 0.5 y τ coherente en VAL/TEST.")


[QA] 22:43:43 Meta OK → 25 features whitelisted: ['fwd_seg_size_min', 'psh_flag_cnt', 'ack_flag_cnt', 'fwd_pkt_len_mean', 'fwd_act_data_pkts', 'fwd_iat_mean', 'bwd_iat_tot', 'idle_mean', 'bwd_pkts_s', 'pkt_len_min']...
[QA] 22:43:44 TRAIN: rows=1,890,770 | prev=27.206% | dtype_ok=True | NaNs=0 | min=0.000000 | max=1.000000 | in[0,1]=True
[QA] 22:43:45 VAL  : rows=541,208 | prev=34.656% | dtype_ok=True | NaNs=0 | min=0.000000 | max=1.000000 | in[0,1]=True
[QA] 22:43:45 TEST : rows=713,747 | prev=46.694% | dtype_ok=True | NaNs=0 | min=0.000000 | max=1.000000 | in[0,1]=True
[QA] 22:43:49 Sanity permutado VAL → AUROC=0.5013 [OK] (esperado ~0.5)
[QA] 22:43:49 τ seleccionado = 0.000045
[QA] 22:44:10 Consistencia VAL vs metrics_val.json → OK
[QA] 22:44:10 Consistencia TEST vs metrics_test.json → OK
[QA] 22:44:10 ✅ QA COMPLETA: sin fuga, integridad OK, sanity permutado ≈ 0.5 y τ coherente en VAL/TEST.


In [ ]:
# =========================================================
# CELDA 5.5 — PREVIEW ESTRATIFICADO (500 VAL y 500 TEST, robusta)
#   • Intenta NO reevaluar: usa X_val, y_val, val_prob, X_test, y_test, test_prob
#   • Si falta algo y existe 'hgb' en memoria, recarga desde disco y recalcula probas
#   • Muestra aleatoria estratificada (mantiene prevalencia del split)
#   • Imprime TODAS las features + probabilidad + pred + label real + acierto
# =========================================================
from __future__ import annotations
from pathlib import Path
import json, numpy as np, pandas as pd
from __future__ import annotations
from sklearn.metrics import confusion_matrix

def _ensure_arrays_ready():
    """Garantiza que X_val, y_val, val_prob, X_test, y_test, test_prob existen y están alineados.
       Prefiere usar los ya en memoria (sin reevaluar). Si falta algo e hgb está disponible,
       recarga desde disco y recalcula probas. Si tampoco hay hgb, lanza un assert claro."""
    global X_val, y_val, val_prob, X_test, y_test, test_prob, BASE, FEATURES, label_col, hgb

    need_names = ["X_val","y_val","val_prob","X_test","y_test","test_prob","BASE"]
    have = {n: (n in globals()) for n in need_names}
    # Si todo existe y no es None → OK
    def _ok(x): return (x is not None)
    if all(have.values()) and _ok(X_val) and _ok(y_val) and _ok(val_prob) and _ok(X_test) and _ok(y_test) and _ok(test_prob):
        return  # todo listo

    # Si falta algo, intentar reconstruir con hgb
    assert "BASE" in globals() and BASE is not None, "Falta BASE; ejecuta Celdas 0–4."
    FEAT_DIR = Path(BASE) / "prepared" / "features"
    # Intenta leer meta para saber FEATURES/label_col si no están
    if "FEATURES" not in globals() or FEATURES is None or "label_col" not in globals():
        meta = json.loads((FEAT_DIR/"features_meta.json").read_text())
        FEATURES_local = meta["features"]
        label_col_local = meta.get("label_col","y")
    else:
        FEATURES_local = FEATURES
        label_col_local = label_col

    def _load_split_df(split: str):
        parts = sorted((FEAT_DIR/split).glob("part_*.parquet")) or sorted((FEAT_DIR/split).glob("*.parquet"))
        assert parts, f"No hay parts en {FEAT_DIR/split}"
        Xs, ys = [], []
        for p in parts:
            df = pd.read_parquet(p, columns=[*FEATURES_local, label_col_local])
            Xs.append(df[FEATURES_local].astype("float32"))
            ys.append(df[label_col_local].astype(np.int8).to_numpy())
        X = pd.concat(Xs, ignore_index=True)
        y = np.concatenate(ys)
        return X, y

    need_recompute_val = ("X_val" not in globals() or X_val is None or
                          "y_val" not in globals() or y_val is None or
                          "val_prob" not in globals() or val_prob is None)
    need_recompute_test = ("X_test" not in globals() or X_test is None or
                           "y_test" not in globals() or y_test is None or
                           "test_prob" not in globals() or test_prob is None)

    if need_recompute_val or need_recompute_test:
        assert "hgb" in globals() and hgb is not None, (
            "Falta algún objeto (X/y/prob) y no está 'hgb' en memoria para recalcular.\n"
            "Solución rápida: vuelve a ejecutar la Celda 5 y después esta 5.5."
        )

    # Reconstruye lo que falte
    if need_recompute_val:
        X_val, y_val = _load_split_df("val")
        val_prob = hgb.predict_proba(X_val)[:,1]
    if need_recompute_test:
        X_test, y_test = _load_split_df("test")
        test_prob = hgb.predict_proba(X_test)[:,1]

    # Alineaciones básicas
    assert len(X_val)==len(y_val)==len(val_prob), f"VAL desalineado: {len(X_val)} vs {len(y_val)} vs {len(val_prob)}"
    assert len(X_test)==len(y_test)==len(test_prob), f"TEST desalineado: {len(X_test)} vs {len(y_test)} vs {len(test_prob)}"

_ensure_arrays_ready()

# ---- Cargar τ elegido (misma Celda 5) ----
MODEL_DIR = Path(BASE) / "models_ids2018"
TAU = float(json.loads((MODEL_DIR/"decision_thresholds.json").read_text())["selected"]["tau"])

# ---- Muestreo estratificado por prevalencia del split ----
def stratified_sample_indices(y: np.ndarray, n_total: int, rng: np.random.Generator):
    prev = float((y == 1).mean())
    n_pos = min(int(round(n_total * prev)), int((y == 1).sum()))
    n_neg = min(n_total - n_pos, int((y == 0).sum()))
    # Relleno si falta por redondeos
    falta = n_total - (n_pos + n_neg)
    if falta > 0:
        disp_pos = int((y == 1).sum()) - n_pos
        disp_neg = int((y == 0).sum()) - n_neg
        if disp_pos >= disp_neg and disp_pos > 0:
            add = min(falta, disp_pos); n_pos += add; falta -= add
        if falta > 0 and disp_neg > 0:
            add = min(falta, disp_neg); n_neg += add; falta -= add

    pos_idx = np.flatnonzero(y == 1)
    neg_idx = np.flatnonzero(y == 0)
    take_pos = rng.choice(pos_idx, size=n_pos, replace=False) if n_pos > 0 else np.array([], dtype=int)
    take_neg = rng.choice(neg_idx, size=n_neg, replace=False) if n_neg > 0 else np.array([], dtype=int)
    idx = np.concatenate([take_pos, take_neg])
    rng.shuffle(idx)
    return idx

def build_preview_df(split_name: str, X: pd.DataFrame, y: np.ndarray, prob: np.ndarray,
                     n_total: int, seed: int) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    n_total = min(n_total, len(y))
    idx = stratified_sample_indices(y, n_total, rng)
    subX = X.iloc[idx].copy()
    sub  = subX.assign(
        probabilidad = prob[idx].astype(float),
        pred         = (prob[idx] >= TAU).astype("int8"),
        **{"label real": y[idx].astype("int8")}
    )
    sub["acierto"] = (sub["pred"] == sub["label real"]).astype("int8")
    sub.insert(0, "split", split_name.upper())
    cols_features = list(subX.columns)
    cols_final = ["split"] + cols_features + ["probabilidad","pred","label real","acierto"]
    return sub[cols_final].reset_index(drop=True)

# ---- Construir e imprimir 500/500 estratificados ----
N_VAL, N_TEST = 500, 500
prev_val  = build_preview_df("val",  X_val,  y_val,  val_prob,  n_total=N_VAL,  seed=2025)
prev_test = build_preview_df("test", X_test, y_test, test_prob, n_total=N_TEST, seed=777)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 220)

print("\n=== VALIDATION (500 aleatorias estratificadas; sin reevaluar si estaba todo listo) ===")
print(prev_val.to_string(index=False, justify="left", float_format=lambda x: f"{x:.6f}"))
print(f"\nDetalle VAL → aciertos: {int(prev_val['acierto'].sum())}/{len(prev_val)} | "
      f"clase 0: {int((prev_val['label real']==0).sum())} | clase 1: {int((prev_val['label real']==1).sum())} | "
      f"τ={TAU:.8f}")

print("\n=== TEST (500 aleatorias estratificadas; sin reevaluar si estaba todo listo) ===")
print(prev_test.to_string(index=False, justify="left", float_format=lambda x: f"{x:.6f}"))
print(f"\nDetalle TEST → aciertos: {int(prev_test['acierto'].sum())}/{len(prev_test)} | "
      f"clase 0: {int((prev_test['label real']==0).sum())} | clase 1: {int((prev_test['label real']==1).sum())} | "
      f"τ={TAU:.8f}")

def metrics_from_preview(df: pd.DataFrame, name: str):
    assert all(c in df.columns for c in ["pred","label real","probabilidad","acierto"]), \
        f"Faltan columnas en {name}. Corre antes la Celda 5.5."
    y_true = df["label real"].to_numpy()
    y_pred = df["pred"].to_numpy()

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    n = len(df)
    # métricas básicas
    acc = (tp + tn) / n if n else np.nan
    prec = tp / (tp + fp) if (tp + fp) else 0.0
    rec  = tp / (tp + fn) if (tp + fn) else 0.0  # = TPR
    f1   = (2*prec*rec)/(prec+rec) if (prec+rec) else 0.0
    tnr  = tn / (tn + fp) if (tn + fp) else 0.0  # specificity
    fpr  = fp / (fp + tn) if (fp + tn) else 0.0
    fnr  = fn / (fn + tp) if (fn + tp) else 0.0
    bal_acc = 0.5*(rec + tnr)

    # métricas complementarias
    npv = tn / (tn + fn) if (tn + fn) else 0.0
    # Matthews Correlation Coefficient (MCC)
    denom = np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    mcc = ((tp*tn - fp*fn) / denom) if denom else 0.0

    # prevalencia y tasa de positivos predichos
    prevalence = (tp + fn) / n if n else np.nan
    pred_pos_rate = (tp + fp) / n if n else np.nan

    # resumen de aciertos/errores
    aciertos = int((df["acierto"] == 1).sum())
    errores  = n - aciertos

    print(f"\n========== MÉTRICAS EXTENDIDAS · {name} (n={n}) ==========")
    print(f"Conteos         → TN={tn} | FP={fp} | FN={fn} | TP={tp}")
    print(f"Aciertos/Errores→ {aciertos}/{errores}")
    print(f"Accuracy        → {acc:.4f}")
    print(f"Precision       → {prec:.4f}   (TP/(TP+FP))")
    print(f"Recall (TPR)    → {rec:.4f}   (TP/(TP+FN))")
    print(f"F1              → {f1:.4f}")
    print(f"Specificity(TNR)→ {tnr:.4f}   (TN/(TN+FP))")
    print(f"FPR             → {fpr:.4f}   (FP/(FP+TN))")
    print(f"FNR             → {fnr:.4f}   (FN/(FN+TP))")
    print(f"Balanced Acc.   → {bal_acc:.4f}")
    print(f"NPV             → {npv:.4f}   (TN/(TN+FN))")
    print(f"MCC             → {mcc:.4f}")
    print(f"Prevalencia muestral        → {prevalence*100:.2f}%")
    print(f"Tasa de positivos predichos → {pred_pos_rate*100:.2f}%")

    # pequeño desglose por etiqueta real
    n0 = int((y_true == 0).sum()); n1 = int((y_true == 1).sum())
    ok0 = tn; ok1 = tp
    err0 = fp; err1 = fn
    print(f"\nDesglose por clase real:")
    print(f"  Clase 0 → total={n0} | aciertos={ok0} | errores(FP)={err0} | %acierto={ok0/max(1,n0)*100:.2f}%")
    print(f"  Clase 1 → total={n1} | aciertos={ok1} | errores(FN)={err1} | %acierto={ok1/max(1,n1)*100:.2f}%")

# Ejecuta para tus previews de 500/500
metrics_from_preview(prev_val,  "VAL (500 estratificadas)")
metrics_from_preview(prev_test, "TEST (500 estratificadas)")



=== VALIDATION (500 aleatorias estratificadas; sin reevaluar si estaba todo listo) ===
split  fwd_seg_size_min  psh_flag_cnt  ack_flag_cnt  fwd_pkt_len_mean  fwd_act_data_pkts  fwd_iat_mean  bwd_iat_tot  idle_mean  bwd_pkts_s  pkt_len_min  totlen_fwd_pkts  urg_flag_cnt  bwd_iat_std  init_fwd_win_byts  bwd_iat_mean  idle_min  flow_pkts_s  fwd_psh_flags  totlen_bwd_pkts  init_bwd_win_byts  fin_flag_cnt  active_mean  active_std  pkt_len_var  flow_byts_s  probabilidad  pred  label real  acierto
VAL   0.727273          0.000000      1.000000      0.000000          0.000000           0.999648      0.000000     0.000000   0.125000    0.000000     0.000000         1.000000      0.000000     0.003693           0.000000      0.000000  0.062500     0.000000       0.000000         0.003525           0.000000      0.000000     0.000000    0.000000     0.000000     0.000000      0     1           0       
VAL   0.454545          0.000000      1.000000      0.007078          0.003279           0.999

In [ ]:
# ================================================================
# CELDA 5.9-FAST (FIX) — Permutation Importance (AUPRC) con muestreo
# ================================================================
import os, json, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import joblib
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

def compute_perm_importance_fast(
    base="/content/drive/MyDrive/TFG/DATASETS/IDS2018",
    sample_rows=120_000,        # súbirlo si se quiere más precisión
    n_repeats=5,                # subirlo si tenemos tiempo
    top_k_plot=15,
    random_state=42
):
    BASE = Path(base)
    FEAT_DIR  = BASE / "prepared" / "features"
    VAL_DIR   = FEAT_DIR / "val"
    MODEL_DIR = BASE / "models_ids2018"

    # --- Meta + modelo
    meta = json.loads((FEAT_DIR/"features_meta.json").read_text())
    FEATURES  = list(meta["features"])
    label_col = meta.get("label_col", "y")
    hgb = joblib.load(MODEL_DIR / "hgb_model.pkl")

    # --- Parts VALIDATION
    parts = sorted(VAL_DIR.glob("part_*.parquet")) or sorted(VAL_DIR.glob("*.parquet"))
    assert parts, f"No encuentro parts en {VAL_DIR}"

    # --- Muestreo estratificado global hasta sample_rows
    rng = np.random.default_rng(random_state)
    sizes, pos_counts = [], []
    for p in parts:
        y = pd.read_parquet(p, columns=[label_col], engine="pyarrow")[label_col].to_numpy()
        sizes.append(len(y)); pos_counts.append(int((y == 1).sum()))
    total = int(sum(sizes))
    target = min(sample_rows, total)

    X_chunks, y_chunks = [], []
    for p, sz in zip(parts, sizes):
        take = int(round(target * (sz / total)))
        if take <= 0:
            continue
        df = pd.read_parquet(p, columns=[*FEATURES, label_col], engine="pyarrow")
        if take < sz:
            y = df[label_col].to_numpy()
            pos_idx = np.flatnonzero(y == 1); neg_idx = np.flatnonzero(y == 0)
            prop_pos = len(pos_idx) / sz if sz else 0.0
            n_pos = min(int(round(take * prop_pos)), len(pos_idx))
            n_neg = min(take - n_pos, len(neg_idx))
            falta = take - (n_pos + n_neg)
            if falta > 0 and (len(neg_idx) - n_neg) >= falta: n_neg += falta
            elif falta > 0 and (len(pos_idx) - n_pos) >= falta: n_pos += falta
            sel = np.concatenate([
                rng.choice(pos_idx, size=n_pos, replace=False) if n_pos>0 else np.array([], int),
                rng.choice(neg_idx, size=n_neg, replace=False) if n_neg>0 else np.array([], int),
            ])
            rng.shuffle(sel)
            df = df.iloc[sel]
        X_chunks.append(df[FEATURES].astype("float32"))
        y_chunks.append(df[label_col].astype("int8").to_numpy())
    Xv = pd.concat(X_chunks, ignore_index=True); yv = np.concatenate(y_chunks)
    print(f"[5.9-FAST] VALIDATION muestreada: X={Xv.shape} | y_pos%={100*(yv==1).mean():.3f}%")

    # --- Permutation Importance (AUPRC) ---
    print(f"[5.9-FAST] permutation_importance (average_precision, n_repeats={n_repeats})…")
    pi = permutation_importance(
        hgb, Xv, yv,
        scoring="average_precision",
        n_repeats=n_repeats,
        random_state=random_state,
        n_jobs=-1
    )
    imp_perm = pd.DataFrame({
        "feature": FEATURES,
        "importance_mean": pi.importances_mean,
        "importance_std":  pi.importances_std,
        "n_repeats": [pi.importances.shape[1]] * len(FEATURES),
        "rows_used": [len(Xv)] * len(FEATURES),
    }).sort_values("importance_mean", ascending=False).reset_index(drop=True)

    out_csv = MODEL_DIR / "perm_importance_val_ap_FAST.csv"
    imp_perm.to_csv(out_csv, index=False)
    print(f"[5.9-FAST] Permutation CSV -> {out_csv}")

    # --- Figura Top-K
    k = min(top_k_plot, len(imp_perm))
    plt.figure(figsize=(8, 0.45*k + 1.5))
    plt.barh(imp_perm.loc[:k-1, "feature"][::-1],
             imp_perm.loc[:k-1, "importance_mean"][::-1])
    plt.xlabel("↓ AUPRC al permutar (media)")
    plt.ylabel("Feature")
    plt.title(f"Permutation importance (VALIDATION, n={len(Xv):,}, repeats={n_repeats})")
    plt.tight_layout()
    out_png = MODEL_DIR / "perm_importance_val_ap_FAST_top15.png"
    plt.savefig(out_png, dpi=160, bbox_inches="tight")
    plt.close()
    print(f"[5.9-FAST] PNG -> {out_png}")

    print("\n[5.9-FAST] Top-10 (Permutation AUPRC):")
    with pd.option_context('display.max_rows', 10):
        print(imp_perm.head(10).to_string(index=False,
              formatters={"importance_mean": "{:.6f}".format,
                          "importance_std":  "{:.6f}".format}))
    return imp_perm

# Ejecuta:
_ = compute_perm_importance_fast(sample_rows=120_000, n_repeats=5)


[5.9-FAST] VALIDATION muestreada: X=(120000, 25) | y_pos%=34.656%
[5.9-FAST] permutation_importance (average_precision, n_repeats=5)…
[5.9-FAST] Permutation CSV -> /content/drive/MyDrive/TFG/DATASETS/IDS2018/models_ids2018/perm_importance_val_ap_FAST.csv
[5.9-FAST] PNG -> /content/drive/MyDrive/TFG/DATASETS/IDS2018/models_ids2018/perm_importance_val_ap_FAST_top15.png

[5.9-FAST] Top-10 (Permutation AUPRC):
          feature importance_mean importance_std  n_repeats  rows_used
 fwd_seg_size_min        0.404460       0.000682          5     120000
fwd_act_data_pkts        0.269511       0.001141          5     120000
      flow_pkts_s        0.164025       0.000778          5     120000
  totlen_fwd_pkts        0.107827       0.000926          5     120000
     fwd_iat_mean        0.059852       0.000449          5     120000
init_bwd_win_byts        0.045949       0.000185          5     120000
      bwd_iat_std        0.037862       0.000213          5     120000
  totlen_bwd_pkts     

In [ ]:
# ================================================================
# CELDA 7 — DEF (Export final con README detallado + artefactos)
#   • Empaqueta modelo, umbral, métricas, meta de features, parámetros del HGB
#   • Añade opcionales si existen: calibrador, perm_importance, metadatos de split/versions
#   • Genera README.md con resumen: τ, objetivos NP, métricas VAL/TEST, prevalencias, ENV, features, uso
#   • Exporta ZIP a export/IDS2018_HGB_export_<TS>.zip
# ================================================================
import os, sys, json, shutil, textwrap
from pathlib import Path
from datetime import datetime

def _num(x):
    try:
        return float(x)
    except Exception:
        return x

# --- Rutas base (usa IDS_BASE fijada en Celda 0) ---
BASE = Path(os.environ.get("IDS_BASE", "/content/drive/MyDrive/TFG/DATASETS/IDS2018")).resolve()
if not BASE.exists():
    print(f"❌ No existe BASE: {BASE}. Ajusta IDS_BASE en Celda 0.")
    sys.exit(0)

PREP        = BASE / "prepared"
FEAT_DIR    = PREP / "features"
SPLITS_DIR  = PREP / "splits"
MODELS_DIR  = BASE / "models_ids2018"
EXPORT_DIR  = BASE / "export"
REPORTS_DIR = BASE / "reports_ids2018"  # por si en el futuro se guarda informes aquí
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

# --- Ficheros principales requeridos ---
required = [
    MODELS_DIR / "hgb_model.pkl",
    MODELS_DIR / "decision_thresholds.json",
    MODELS_DIR / "metrics_val.json",
    MODELS_DIR / "metrics_test.json",
    FEAT_DIR   / "features_meta.json",
]
missing_req = [str(p) for p in required if not p.exists()]
if missing_req:
    print("❌ Faltan ficheros requeridos para exportar:")
    for m in missing_req: print("   -", m)
    sys.exit(0)

# --- Ficheros opcionales (se añaden si existen) ---
optional = [
    MODELS_DIR / "hgb_params.json",                 # params del HGB
    MODELS_DIR / "hgb_calibrator.pkl",              # calibrador si lo hubieras entrenado
    MODELS_DIR / "perm_importance_val_ap.csv",      # importancia por permutación (AUPRC)
    MODELS_DIR / "perm_importance_val_ap_top20.png",
    PREP       / "versions.json",
    PREP       / "settings.json",
    PREP       / "split_meta.json",
    PREP       / "salt_pick.json",
    SPLITS_DIR / "_SPLIT_ROWS_META.json",          # prevalencias/tamaños reales por split
]

# --- Localizar último manifest de inferencia (cualquier split), si existe ---
manifests = sorted(PREP.glob("inference/*/*/inference_manifest.json"),
                   key=lambda p: p.stat().st_mtime if p.exists() else 0,
                   reverse=True)
last_manifest = manifests[0] if manifests else None

# --- Staging temporal ---
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
stage = EXPORT_DIR / f"_stage_{ts}"
stage.mkdir(parents=True, exist_ok=True)

def cp(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy2(src, dst)

# Copiar requeridos manteniendo estructura relativa a BASE
for f in required:
    cp(f, stage / f.relative_to(BASE))

# Copiar opcionales si existen
present_optional = []
for f in optional:
    if f.exists():
        cp(f, stage / f.relative_to(BASE))
        present_optional.append(f)

# Copiar último manifest (si hay)
if last_manifest:
    cp(last_manifest, stage / "inference_manifest.json")

# --- Cargar info para el README ---
# ENV (los que fijaste en 4-POST/5)
ENV_SNAPSHOT = {
    "IDS_MAX_FPR":   os.getenv("IDS_MAX_FPR"),
    "IDS_MIN_TPR":   os.getenv("IDS_MIN_TPR"),
    "IDS_NP_MARGIN": os.getenv("IDS_NP_MARGIN"),
    "IDS_FAST":      os.getenv("IDS_FAST"),
    "IDS_FAST_ROWS": os.getenv("IDS_FAST_ROWS"),
    "OMP_NUM_THREADS": os.getenv("OMP_NUM_THREADS"),
}

# Decision / τ y objetivos NP
decision = json.loads((MODELS_DIR/"decision_thresholds.json").read_text())
tau       = _num(decision["selected"]["tau"])
val_fpr   = _num(decision["selected"].get("val_fpr", None))
val_tpr   = _num(decision["selected"].get("val_tpr", None))
val_f1    = _num(decision["selected"].get("val_f1", None))
max_fpr   = _num(decision["selected"].get("max_fpr_target", None))
min_tpr   = _num(decision["selected"].get("min_tpr_target", None))
np_margin = _num(decision["selected"].get("np_margin", None))

# Métricas VAL/TEST (detalladas)
m_val  = json.loads((MODELS_DIR/"metrics_val.json").read_text())
m_test = json.loads((MODELS_DIR/"metrics_test.json").read_text())

def _fmt_cm(cm_dict):
    return f"tn={cm_dict['confusion']['tn']}  fp={cm_dict['confusion']['fp']}  fn={cm_dict['confusion']['fn']}  tp={cm_dict['confusion']['tp']}"

# Meta de features y lista
feat_meta = json.loads((FEAT_DIR/"features_meta.json").read_text())
features  = list(feat_meta.get("features", []))
label_col = feat_meta.get("label_col", "y")

# Prevalencias/tamaños por split (si hay meta de splits)
prev_lines = []
split_rows_meta = SPLITS_DIR / "_SPLIT_ROWS_META.json"
if split_rows_meta.exists():
    sm = json.loads(split_rows_meta.read_text())
    for s in ("train","val","test"):
        if s in sm["by_split"]:
            b = sm["by_split"][s]
            prev_lines.append(
                f"- {s.upper():5s}: filas={b['rows']:,} | prev={b['prev']*100:.3f}% | pos={b['pos']:,} | neg={b['neg']:,} | files={b['files']}"
            )

# Parámetros del HGB (si los guardaste)
hgb_params = None
if (MODELS_DIR/"hgb_params.json").exists():
    try:
        hgb_params = json.loads((MODELS_DIR/"hgb_params.json").read_text())
    except Exception:
        hgb_params = None

# Importancia por permutación (si existe CSV)
pi_summary = None
pi_csv = MODELS_DIR / "perm_importance_val_ap.csv"
if pi_csv.exists():
    try:
        import pandas as pd
        df_pi = pd.read_csv(pi_csv)
        topk = df_pi.sort_values("importance_mean", ascending=False).head(10)
        pi_summary = "\n".join(
            f"  {i+1:02d}. {row['feature']}: ΔAUPRC={row['importance_mean']:.6f} (±{row['importance_std']:.6f})"
            for i, row in topk.reset_index(drop=True).iterrows()
        )
    except Exception:
        pi_summary = None

# --- README (informativo y reproducible) ---
readme_md = f"""# Export IDS2018 · HGB (Neyman–Pearson) — {ts}

Este paquete contiene el **modelo final** y los **artefactos** necesarios para reproducir y auditar la evaluación.

## Contenido principal
- `models_ids2018/hgb_model.pkl` — Modelo `HistGradientBoostingClassifier`.
- `models_ids2018/hgb_params.json` — Parámetros del HGB.{" (incluido)" if (MODELS_DIR/'hgb_params.json').exists() else " (no disponible)"}
- `models_ids2018/decision_thresholds.json` — Umbral seleccionado (τ) con objetivos NP.
- `models_ids2018/metrics_val.json` — Métricas en VALIDATION con τ.
- `models_ids2018/metrics_test.json` — Métricas en TEST con el **mismo τ**.
- `prepared/features/features_meta.json` — Lista de features (25 whitelisted), dtype y label.
- `prepared/splits/_SPLIT_ROWS_META.json` — Tamaños y prevalencias reales por split.{" (incluido)" if split_rows_meta.exists() else " (no disponible)"}
- `models_ids2018/perm_importance_val_ap.csv` — Importancia por permutación (AUPRC).{" (incluido)" if pi_csv.exists() else " (no disponible)"}
- `models_ids2018/perm_importance_val_ap_top20.png` — Gráfico Top-20 por AUPRC.{" (incluido)" if (MODELS_DIR/'perm_importance_val_ap_top20.png').exists() else " (no disponible)"}
- `inference_manifest.json` — Último manifiesto de inferencia (si hubo).{" (incluido)" if last_manifest else " (no disponible)"}

## Umbral y objetivos (Neyman–Pearson)
- τ (threshold usado): **{tau:.8f}**
- Objetivo MAX FPR (VALIDATION): **{max_fpr}**
- Mínimo TPR (VALIDATION): **{min_tpr}**
- NP_MARGIN (nota): **{np_margin}**

> τ se seleccionó **en VALIDATION** cumpliendo FPR ≤ objetivo (y TPR ≥ mínimo cuando aplica) y **se aplicó tal cual a TEST**, sin re-ajustes.

## Métricas con τ
**VALIDATION**
- FPR={m_val.get('FPR'):.4f} | TPR/Recall={m_val.get('TPR'):.4f} | Precision={m_val.get('Precision'):.4f} | F1={m_val.get('F1'):.4f}
- AUROC={m_val.get('AUROC'):.4f} | AUPRC={m_val.get('AUPRC'):.4f}
- CM: {_fmt_cm(m_val)}

**TEST**
- FPR={m_test.get('FPR'):.4f} | TPR/Recall={m_test.get('TPR'):.4f} | Precision={m_test.get('Precision'):.4f} | F1={m_test.get('F1'):.4f}
- AUROC={m_test.get('AUROC'):.4f} | AUPRC={m_test.get('AUPRC'):.4f}
- CM: {_fmt_cm(m_test)}

## Prevalencias y tamaños por split
{chr(10).join(prev_lines) if prev_lines else "- (no disponible)"}

## Features (25 whitelisted)
Label column: `{label_col}`
Total: **{len(features)}**
{", ".join(features)}

## Parámetros del modelo (HGB)
{json.dumps(hgb_params, indent=2) if hgb_params else "- (no disponible)"}

## Importancia por permutación (VALIDATION, métrica AUPRC)
{pi_summary if pi_summary else "- (no disponible o no pudo resumirse)"}

## ENV usados (snapshot)
{json.dumps({k:v for k,v in ENV_SNAPSHOT.items() if v is not None}, indent=2)}

## Cómo inferir (pseudo-código)
1. Cargar `hgb_model.pkl`.
2. Cargar `prepared/features/features_meta.json` para el **orden** y **nombres** exactos de las 25 features.
3. Asegurar que las features están **escaladas [0,1]** como en TRAIN (min-max + imputación media de TRAIN; ver Celda 4).
4. Obtener `prob = model.predict_proba(X)[:,1]`.
5. Cargar `decision_thresholds.json` y aplicar `τ`:
   `pred = (prob >= τ).astype(int)`

**Nota sobre calibración:** en este export **no se incluye calibrador**; las probabilidades provienen del HGB nativo. Si tu despliegue requiere cortes por riesgo (no solo 0/1), considera calibrar con `CalibratedClassifierCV` (sigmoid/isotónica) en VALIDATION.

—
*Generado el {ts}*
"""

(stage / "README.md").write_text(readme_md, encoding="utf-8")

# --- Empaquetado ZIP ---
zipname = EXPORT_DIR / f"IDS2018_HGB_export_{ts}.zip"
shutil.make_archive(str(zipname.with_suffix("")), "zip", root_dir=stage)

print("✅ Export listo:", zipname)
print("   Incluidos requeridos y opcionales presentes. README.md con resumen en el ZIP.")


✅ Export listo: /content/drive/MyDrive/TFG/DATASETS/IDS2018/export/IDS2018_HGB_export_20251020_224453.zip
   Incluidos requeridos y opcionales presentes. README.md con resumen en el ZIP.


In [ ]:
# --- Exportar matriz de confusión (VAL y TEST) como imagen PNG
import os, json
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

BASE = Path(os.environ.get("IDS_BASE", "/content/drive/MyDrive/TFG/DATASETS/IDS2018"))
M = BASE / "models_ids2018"
for split in ["val","test"]:
    m = json.loads((M / f"metrics_{split}.json").read_text())
    tn, fp, fn, tp = [m["confusion"][k] for k in ("tn","fp","fn","tp")]
    CM = np.array([[tn, fp],[fn, tp]])

    fig, ax = plt.subplots()
    im = ax.imshow(CM, cmap="Blues")
    for i in range(2):
        for j in range(2):
            ax.text(j, i, f"{CM[i,j]:,}", ha="center", va="center")
    ax.set_xticks([0,1]); ax.set_yticks([0,1])
    ax.set_xticklabels(["Pred 0","Pred 1"]); ax.set_yticklabels(["Real 0","Real 1"])
    ax.set_xlabel("Predicció"); ax.set_ylabel("Real")
    ax.set_title(f"Confusion Matrix — {split.upper()}")
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    out = M / f"cm_{split}.png"
    fig.savefig(out, dpi=160, bbox_inches="tight")
    plt.close(fig)
    print("✔️", out)


✔️ /content/drive/MyDrive/TFG/DATASETS/IDS2018/models_ids2018/cm_val.png
✔️ /content/drive/MyDrive/TFG/DATASETS/IDS2018/models_ids2018/cm_test.png
